In [1]:
import pandas as pd

In [32]:
import numpy as np
from sklearn.model_selection import train_test_split

In [127]:
import torch.utils.data as Data

In [186]:
train_data = pd.read_csv("./tianchi_datasets/track3_round1_train.tsv", sep = "\t", header = None,
                        names = ["sentence1", "sentence2", "labels"])
test_data = pd.read_csv("./tianchi_datasets/track3_round1_testA.tsv", sep = "\t", header = None,
                        names = ["sentence1", "sentence2"])

In [187]:
train_data["sentence1"] = train_data["sentence1"].map(lambda x: [int(x) for x in x.split(" ")])
train_data["sentence2"] = train_data["sentence2"].map(lambda x: [int(x) for x in x.split(" ")])
test_data["sentence1"] = test_data["sentence1"].map(lambda x: [int(x) for x in x.split(" ")])
test_data["sentence2"] = test_data["sentence2"].map(lambda x: [int(x) for x in x.split(" ")])

In [175]:
def get_vocab_v2i_i2v(train_data, test_data):
    vocab = set()
    for data in [train_data, test_data]:
        for n in ["sentence1", "sentence2"]:
            for i in range(len(data[n])):
                vocab.update(data[n][i])
    
    v2i = {v:i for i, v in enumerate(sorted(vocab), start = 1)}
    v2i['[PAD]'] = 0
    v2i['[CLS]'] = len(v2i)
    v2i['[SEP]'] = len(v2i)
    i2v = {i:v for v, i in v2i.items()}
    
    return vocab, v2i, i2v

In [188]:
vocab, v2i, i2v = get_vocab_v2i_i2v(train_data, test_data)

In [226]:
max_len = max([len(s1) + len(s2) + 3  for s1, s2 in zip(
    train_data["sentence1"].tolist() + test_data["sentence1"].tolist(), 
    train_data["sentence2"].tolist() + test_data["sentence2"].tolist())])

In [228]:
train_data["document"] = [[v2i['[CLS]']] + train_data["sentence1"][i] + [v2i['[SEP]']] + 
        train_data["sentence2"][i] + [v2i['[SEP]']] for i in range(len(train_data))]
test_data["document"] = [[v2i['[CLS]']] + test_data["sentence1"][i] + [v2i['[SEP]']] + 
        test_data["sentence2"][i] + [v2i['[SEP]']] for i in range(len(test_data))]

In [229]:
test_size = 0.1
seed_val = 2021
X_train, X_val, y_train, y_val = train_test_split(
            train_data["document"], train_data["labels"], test_size=test_size,
            stratify= train_data["labels"], random_state=seed_val)

In [230]:
def convert_to_bert_datasets(data, labels, max_length):
    input_ids = []
    
    for x in data:
        padded = np.full(max_length, fill_value = 0, dtype = np.int32)
        padded[:len(x)] = x
        input_ids.append(torch.LongTensor([padded]))
    
    input_ids = torch.cat(input_ids, dim = 0)
    
    if labels is None:
        return Data.TensorDataset(input_ids)
    else:
        labels = torch.tensor(labels.values)
        return Data.TensorDataset(input_ids, labels)

In [231]:
train = convert_to_bert_datasets(X_train, y_train, max_len)

In [232]:
validation = convert_to_bert_datasets(X_val, y_val, max_len)

In [233]:
test = convert_to_bert_datasets(test_data["document"], None, max_len)

In [150]:
batch_size = 4
train_loader = Data.DataLoader(train, batch_size = batch_size, shuffle = False)

In [234]:
test_dataloader = Data.DataLoader(test, batch_size = batch_size, shuffle = False)

In [2]:
slices = list(range(1, 7))

In [15]:
def cal_max(nums):
            n = len(nums)
            if n <= 2:
                return max(nums)
            choose = (n+1) // 3
            dp = [[0 for _ in range(choose+1)] for _ in range(n+1)] 
            for i in range(1, n+1):
                for j in range(1, choose+1):
                    dp[i][j] = max((dp[i-2][j-1] if i >= 2 else 0) + nums[i-1], dp[i-1][j])
                    print(dp[i][j])
                    
            return dp[n][choose]

In [18]:
def maxSizeSlices(slices):
        
        def cal_max(nums):
            n = len(nums)
            if n <= 2:
                return max(nums)
            choose = (n+1) // 3
            dp = [[0 for _ in range(choose+1)] for _ in range(n+1)] 
            for i in range(1, n+1):
                for j in range(1, choose+1):
                    dp[i][j] = max((dp[i-2][j-1] if i >= 2 else 0) + nums[i-1], dp[i-1][j])
            
            return dp[n][choose]

        n = len(slices)
        if n <= 3:
            return max(slices)
        
        return max(cal_max(slices[1:]), cal_max(slices[:-1]))

In [19]:
maxSizeSlices(slices)

8

In [20]:
cal_max(slices[1:])

2
2
3
3
4
6
5
8
6
10


10

In [237]:
list([1])

[1]

In [242]:
a = [1]

In [245]:
a = np.array([1])[np.newaxis, :]

In [246]:
a

array([[1]])

In [244]:
a[np.newaxis, :]

TypeError: list indices must be integers or slices, not tuple

In [45]:
from sklearn.model_selection import train_test_split

In [97]:
X_train, X_val, y_train, y_val = train_test_split(
            train_data["document"], train_data["class"], test_size=test_size,
            stratify= train_data["class"], random_state=seed_val)

In [47]:
import torch.utils.data as Data

In [98]:
max_sum = max([len(x) for x in X_train])

In [ ]:
X_train = [[v2i['[CLS]']] + data + []]

In [66]:
model = BertModel.from_pretrained("./pretrain_model/chinese_roberta_wwm_ext")

In [28]:
train_data["document"].map(lambda x: [int(x) for x in x.split(" ")])

0                   [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 4, 11]
1                         [12, 13, 14, 15, 12, 15, 11, 16]
2        [17, 18, 12, 19, 20, 21, 22, 23, 24, 12, 23, 2...
3                 [28, 29, 30, 31, 11, 32, 33, 34, 30, 31]
4                         [29, 35, 36, 29, 29, 37, 36, 29]
                               ...                        
99995    [12, 19, 1162, 126, 53, 66, 12, 19, 79, 389, 1...
99996    [275, 552, 553, 433, 881, 338, 1104, 101, 202,...
99997    [421, 330, 62, 12, 80, 81, 82, 76, 202, 62, 12...
99998    [177, 455, 456, 3474, 964, 1364, 55, 1364, 133...
99999    [29, 168, 12, 19, 1003, 719, 23, 29, 263, 276,...
Name: document, Length: 100000, dtype: object

In [81]:
from transformers import BertTokenizer

In [82]:
tokenizer = BertTokenizer.from_pretrained("./pretrain_model/chinese_roberta_wwm_ext")

In [84]:
len(tokenizer)

21128

In [36]:
from transformers import BertModel

In [15]:
tokenizer("1 2 3")

{'input_ids': [101, 122, 123, 124, 102], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

In [16]:
tokenizer.convert_ids_to_tokens([101, 122, 123, 124, 102])

['[CLS]', '1', '2', '3', '[SEP]']

In [ ]:
model = 

In [35]:
np.unique(train_data["class"], return_counts = True)

(array([0, 1], dtype=int64), array([63564, 36436], dtype=int64))

In [38]:
import torch

In [42]:
torch.ones(2, 2, 2).long()

tensor([[[1, 1],
         [1, 1]],

        [[1, 1],
         [1, 1]]])

In [140]:
tokenizer("我我哦", return_tensors= "pt")

{'input_ids': tensor([[ 101, 2769, 2769, 1521,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [248]:
torch.LongTensor([[1]])

tensor([[1]])

In [249]:
from Data_generator import Corpus

In [250]:
batch_size = 16
seed_val = 1

In [253]:
a = np.array([[0.3, 0.7],[0.2, 0.8]])

In [259]:
a[:, 1]

array([0.7, 0.8])

In [1]:
from Data_generator import Corpus
batch_size = 16
seed_val = 1
corpus = Corpus(batch_size, seed_val)
train_loader, valid_loader, test_loader = corpus.get_loaders()
for batch in train_loader:
    print(batch[:-1])
    print(batch[-1])
    break

********************************Data_precessing*********************************
## dataset size is 100000
*******************************Load  dataset ....*******************************
## Load Datasets Consume 0:00:10 s ###
************************All Train and Test Data loaded !************************
[tensor([[20601,    45,    46,  ...,     0,     0,     0],
        [20601,  1038,   591,  ...,     0,     0,     0],
        [20601,  3299,   249,  ...,     0,     0,     0],
        ...,
        [20601,    87,   956,  ...,     0,     0,     0],
        [20601,  5553,   578,  ...,     0,     0,     0],
        [20601,   670,   217,  ...,     0,     0,     0]])]
tensor([0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1])


In [10]:
batch[:-1][0]

tensor([[20601,    45,    46,  ...,     0,     0,     0],
        [20601,  1038,   591,  ...,     0,     0,     0],
        [20601,  3299,   249,  ...,     0,     0,     0],
        ...,
        [20601,    87,   956,  ...,     0,     0,     0],
        [20601,  5553,   578,  ...,     0,     0,     0],
        [20601,   670,   217,  ...,     0,     0,     0]])

In [3]:
batch[0]

tensor([[20601,    45,    46,  ...,     0,     0,     0],
        [20601,  1038,   591,  ...,     0,     0,     0],
        [20601,  3299,   249,  ...,     0,     0,     0],
        ...,
        [20601,    87,   956,  ...,     0,     0,     0],
        [20601,  5553,   578,  ...,     0,     0,     0],
        [20601,   670,   217,  ...,     0,     0,     0]])

In [65]:
from transformers import BertModel

In [ ]:
model(*batch[:-1])

In [48]:
import numpy as np
y_test = np.array([0,0,1,1])
y_pred1 = np.array([0.3,0.2,0.25,0.7])
y_pred2 = np.array([0,0,1,0]) 
from sklearn.metrics import roc_auc_score 
# 预测值是概率
auc_score1 = roc_auc_score(y_test,y_pred1)
print(auc_score1)

0.75


In [42]:
logits = np.random.rand(16, 2)

In [34]:
logits.shape

(4, 2)

In [25]:
logits[:,1]

array([0.47471689, 0.76443855, 0.73893367, 0.46974965])

In [43]:
y_pred = np.array([])
y_pred = np.append(y_pred, logits[:,1])

In [44]:
y_pred

array([5.82289618e-01, 6.56078992e-01, 7.94656512e-04, 3.93339930e-01,
       2.82572855e-01, 9.58476154e-01, 2.31650849e-01, 6.05707484e-01,
       8.09247471e-01, 7.02316260e-01, 1.88429106e-02, 7.00293130e-01,
       9.72872622e-01, 7.89888877e-01, 7.48251878e-01, 8.73607196e-02])

In [35]:
y_pred.shape

(4,)

In [37]:
import torch

In [38]:
a = torch.tensor([0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1])

In [41]:
b = a.data.numpy()

In [47]:
from sklearn.metrics import f1_score

In [49]:
f1_score(y_test, y_pred2)

0.6666666666666666

In [50]:
f1_score(y_pred2, y_test)

0.6666666666666666

In [51]:
2813 * 32

90016

In [197]:
from imp import reload

In [203]:
import Data_generator

In [204]:
reload(Data_generator)

<module 'Data_generator' from 'E:\\lv_python\\NLP\\阿里天池小布助手\\Data_generator.py'>

In [205]:
from Data_generator import Corpus

In [206]:
batch_size = 4
seed_val = 2021
corpus = Corpus(batch_size, seed_val)

********************************Data_precessing*********************************
## dataset size is 100000
*******************************Load  dataset ....*******************************
## Load Datasets Consume 0:00:10 s ###
************************All Train and Test Data loaded !************************


In [207]:
train_dataloader, valid_dataloader, test_dataloader = corpus.get_loaders()

In [62]:
x[0][0].size(0)

93

In [64]:
for x in test_dataloader:
    if x[0][0].size(0) != 93: 
        print("bug")

In [79]:
from tqdm import tqdm

In [208]:
for y in tqdm(test_dataloader):
    try:
        outputs = model(*y)
    except:
        print(y)
        break

  8%|██████▏                                                                    | 2063/25000 [12:23<2:17:42,  2.78it/s]

[tensor([[20165,    12,   126,  1758,    66,   444, 20166,    12,   444,    28,
             9,  1758,    76, 20166,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0]])]


In [85]:
y

[tensor([[20601,   200,  1789,    52,    52,  6191,   433,  1652,    52,    11,
            415,     8,   344,     9,  5192,   267, 20602,  2320,  2570,  1104,
             52,  1405, 21600,  1382,   173, 20602,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0]])]

In [76]:
model(*y)[0][:, 0, :].size()

torch.Size([1, 768])

In [68]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(21128, 768, padding_idx=1)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1)
          )
        )
        (in

In [87]:
import pandas as pd

In [186]:
train_data = pd.read_csv("./tianchi_datasets/track3_round1_train.tsv", sep="\t", header=None,
                                 names=["sentence1", "sentence2", "labels"])
test_data = pd.read_csv("./tianchi_datasets/track3_round1_testA.tsv", sep="\t", header=None,
                                names=["sentence1", "sentence2"])

In [187]:
def convrt_str_to_list(train_data, test_data):
    for n in  ["sentence1", "sentence2"]:
        for m in [train_data, test_data]:
                    m[n] = m[n].map(lambda x: [int(x) for x in x.split(" ")])

In [188]:
convrt_str_to_list(train_data, test_data)

In [120]:
vocab = set()
for data in [train_data, test_data]:
    for n in ["sentence1", "sentence2"]:
        for i in range(len(data[n])):
            regular(data[n][i])
            vocab.update(data[n][i])

In [189]:
def regular(data):
    ## 因为bert的vocab size最大为21128,因此将大于21128的全部替换为0，['PAD']
    for i in range(len(data)):
        num = data[i]
        data[i] = num if num < 21125 else 0

In [190]:
def get_vocab_v2i_i2v(train_data, test_data):
    ## 获得语料库vocab, idx2label, label2idx
    vocab = set()
    for data in [train_data, test_data]:
        for n in ["sentence1", "sentence2"]:
            for i in range(len(data[n])):
                regular(data[n][i])
                vocab.update(data[n][i])


    v2i = {v: i for i, v in enumerate(sorted(vocab))}
    v2i['[PAD]'] = v2i.pop(0)
    v2i['[CLS]'] = len(v2i)
    v2i['[SEP]'] = len(v2i)
    i2v = {i: v for v, i in v2i.items()}

    return vocab, v2i, i2v

In [191]:
vocab1, v2i1, i2v1 = get_vocab_v2i_i2v(train_data, test_data)

In [193]:
v2i['[PAD]']

0

In [192]:
vocab1

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [150]:
vocab = set()
for data in [train_data, test_data]:
    for n in ["sentence1", "sentence2"]:
        for i in range(len(data[n])):
            regular(data[n][i])
            vocab.update(data[n][i])

In [167]:
v2i[21124]

20164

In [168]:
i2v[20164]

21124

In [152]:
v2i = {v: i for i, v in enumerate(sorted(vocab))}

In [155]:
v2i['[PAD]'] = v2i.pop(0) 

In [156]:
v2i['[PAD]']

0

In [157]:
v2i['[CLS]'] = len(v2i)
v2i['[SEP]'] = len(v2i)
i2v = {i: v for v, i in v2i.items()}

In [164]:
sorted(list(vocab), reverse = True)

[21124,
 21122,
 21120,
 21118,
 21115,
 21113,
 21109,
 21108,
 21106,
 21105,
 21104,
 21103,
 21101,
 21099,
 21098,
 21094,
 21093,
 21092,
 21091,
 21090,
 21085,
 21084,
 21083,
 21081,
 21080,
 21078,
 21077,
 21076,
 21075,
 21073,
 21072,
 21070,
 21069,
 21068,
 21067,
 21065,
 21062,
 21059,
 21055,
 21051,
 21050,
 21048,
 21039,
 21034,
 21032,
 21023,
 21020,
 21019,
 21017,
 21014,
 21012,
 21009,
 21006,
 21005,
 21003,
 21002,
 21001,
 20997,
 20996,
 20995,
 20992,
 20991,
 20984,
 20981,
 20979,
 20978,
 20977,
 20976,
 20975,
 20974,
 20973,
 20972,
 20971,
 20961,
 20960,
 20959,
 20958,
 20957,
 20956,
 20955,
 20954,
 20953,
 20949,
 20948,
 20947,
 20946,
 20943,
 20942,
 20939,
 20936,
 20935,
 20933,
 20931,
 20930,
 20929,
 20928,
 20927,
 20924,
 20923,
 20921,
 20919,
 20918,
 20916,
 20915,
 20913,
 20911,
 20909,
 20908,
 20907,
 20905,
 20904,
 20903,
 20902,
 20900,
 20895,
 20893,
 20890,
 20887,
 20885,
 20880,
 20879,
 20877,
 20875,
 20872,
 20871,


In [169]:
21124 in v2i.keys()

True

In [162]:
sorted(i2v.keys(),reverse = True)

[20166,
 20165,
 20164,
 20163,
 20162,
 20161,
 20160,
 20159,
 20158,
 20157,
 20156,
 20155,
 20154,
 20153,
 20152,
 20151,
 20150,
 20149,
 20148,
 20147,
 20146,
 20145,
 20144,
 20143,
 20142,
 20141,
 20140,
 20139,
 20138,
 20137,
 20136,
 20135,
 20134,
 20133,
 20132,
 20131,
 20130,
 20129,
 20128,
 20127,
 20126,
 20125,
 20124,
 20123,
 20122,
 20121,
 20120,
 20119,
 20118,
 20117,
 20116,
 20115,
 20114,
 20113,
 20112,
 20111,
 20110,
 20109,
 20108,
 20107,
 20106,
 20105,
 20104,
 20103,
 20102,
 20101,
 20100,
 20099,
 20098,
 20097,
 20096,
 20095,
 20094,
 20093,
 20092,
 20091,
 20090,
 20089,
 20088,
 20087,
 20086,
 20085,
 20084,
 20083,
 20082,
 20081,
 20080,
 20079,
 20078,
 20077,
 20076,
 20075,
 20074,
 20073,
 20072,
 20071,
 20070,
 20069,
 20068,
 20067,
 20066,
 20065,
 20064,
 20063,
 20062,
 20061,
 20060,
 20059,
 20058,
 20057,
 20056,
 20055,
 20054,
 20053,
 20052,
 20051,
 20050,
 20049,
 20048,
 20047,
 20046,
 20045,
 20044,
 20043,
 20042,


In [147]:
v2i[0]

'[PAD]'

In [148]:
v2i.pop(0)

'[PAD]'

In [146]:
v2i['[PAD]']

0

In [128]:
v2i = {v: i for i, v in enumerate(sorted(vocab))}

In [129]:
v2i

{0: 0,
 1: 1,
 2: 2,
 3: 3,
 4: 4,
 5: 5,
 6: 6,
 7: 7,
 8: 8,
 9: 9,
 10: 10,
 11: 11,
 12: 12,
 13: 13,
 14: 14,
 15: 15,
 16: 16,
 17: 17,
 18: 18,
 19: 19,
 20: 20,
 21: 21,
 22: 22,
 23: 23,
 24: 24,
 25: 25,
 26: 26,
 27: 27,
 28: 28,
 29: 29,
 30: 30,
 31: 31,
 32: 32,
 33: 33,
 34: 34,
 35: 35,
 36: 36,
 37: 37,
 38: 38,
 39: 39,
 40: 40,
 41: 41,
 42: 42,
 43: 43,
 44: 44,
 45: 45,
 46: 46,
 47: 47,
 48: 48,
 49: 49,
 50: 50,
 51: 51,
 52: 52,
 53: 53,
 54: 54,
 55: 55,
 56: 56,
 57: 57,
 58: 58,
 59: 59,
 60: 60,
 61: 61,
 62: 62,
 63: 63,
 64: 64,
 65: 65,
 66: 66,
 67: 67,
 68: 68,
 69: 69,
 70: 70,
 71: 71,
 72: 72,
 73: 73,
 74: 74,
 75: 75,
 76: 76,
 77: 77,
 78: 78,
 79: 79,
 80: 80,
 81: 81,
 82: 82,
 83: 83,
 84: 84,
 85: 85,
 86: 86,
 87: 87,
 88: 88,
 89: 89,
 90: 90,
 91: 91,
 92: 92,
 93: 93,
 94: 94,
 95: 95,
 96: 96,
 97: 97,
 98: 98,
 99: 99,
 100: 100,
 101: 101,
 102: 102,
 103: 103,
 104: 104,
 105: 105,
 106: 106,
 107: 107,
 108: 108,
 109: 109,
 110: 110,

In [109]:
total_cnt

1607317

In [121]:
vocab_list = sorted(list(vocab), reverse = True)

In [122]:
vocab_list

[21124,
 21122,
 21120,
 21118,
 21115,
 21113,
 21109,
 21108,
 21106,
 21105,
 21104,
 21103,
 21101,
 21099,
 21098,
 21094,
 21093,
 21092,
 21091,
 21090,
 21085,
 21084,
 21083,
 21081,
 21080,
 21078,
 21077,
 21076,
 21075,
 21073,
 21072,
 21070,
 21069,
 21068,
 21067,
 21065,
 21062,
 21059,
 21055,
 21051,
 21050,
 21048,
 21039,
 21034,
 21032,
 21023,
 21020,
 21019,
 21017,
 21014,
 21012,
 21009,
 21006,
 21005,
 21003,
 21002,
 21001,
 20997,
 20996,
 20995,
 20992,
 20991,
 20984,
 20981,
 20979,
 20978,
 20977,
 20976,
 20975,
 20974,
 20973,
 20972,
 20971,
 20961,
 20960,
 20959,
 20958,
 20957,
 20956,
 20955,
 20954,
 20953,
 20949,
 20948,
 20947,
 20946,
 20943,
 20942,
 20939,
 20936,
 20935,
 20933,
 20931,
 20930,
 20929,
 20928,
 20927,
 20924,
 20923,
 20921,
 20919,
 20918,
 20916,
 20915,
 20913,
 20911,
 20909,
 20908,
 20907,
 20905,
 20904,
 20903,
 20902,
 20900,
 20895,
 20893,
 20890,
 20887,
 20885,
 20880,
 20879,
 20877,
 20875,
 20872,
 20871,


In [100]:
len(vocab_list)

20600

In [104]:
v2i = {v: i for i, v in enumerate(sorted(vocab), start=1)}

In [142]:
tokenizer.vocab['[PAD]']

0

In [112]:
def regular()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [117]:
a = {"a":1, "c":2}
#方法
a["b"]=a.pop("a")

In [118]:
a

{'c': 2, 'b': 1}

In [220]:
predicts = np.array([1.32131351465, 2.5456546464, 3.65465465464])

In [211]:
for predict in predicts:
    print(predict)

1.0
2.0
3.0


In [215]:
type(str(predict))

str

In [221]:
with open("aa.tsv", "w") as fw:
    for predict in predicts:
        fw.write(str(round(predict, 3)))
        fw.write("\n")

In [227]:
input_ids = tokenizer("202010 113 222222")
input_ids["input_ids"]

[101, 8439, 8311, 8989, 9918, 8683, 8144, 102]

In [228]:
tokenizer.convert_ids_to_tokens(input_ids["input_ids"])

['[CLS]', '2020', '##10', '113', '222', '##22', '##2', '[SEP]']

In [1]:
import pandas as pd

In [75]:
data = pd.read_csv("./tianchi_datasets/track3_round1_train.tsv", sep="\t", header=None,
                            names=["sentence1", "sentence2", "labels"])

In [3]:
from sklearn.model_selection import StratifiedKFold

In [4]:
n_folds = 10
seed_val = 2021
skf = StratifiedKFold(n_splits = n_folds, shuffle = True, random_state = seed_val)

In [7]:
import numpy as np

In [8]:
np.unique(train_data["labels"], return_counts = True)

(array([0, 1], dtype=int64), array([63564, 36436], dtype=int64))

In [13]:
63564 / 36436

1.7445383686463938

In [20]:
train_data["sentence1"].iloc[[0, 2]]

0                 1 2 3 4 5 6 7
2    17 18 12 19 20 21 22 23 24
Name: sentence1, dtype: object

In [17]:
for train_index, test_index in skf.split(train_data, train_data["labels"]):
    X_train, X_val = train_data.iloc[train_index], train_data.iloc[test_index]
    

(array([0, 1], dtype=int64), array([57207, 32793], dtype=int64))
(array([0, 1], dtype=int64), array([6357, 3643], dtype=int64))
(array([0, 1], dtype=int64), array([57207, 32793], dtype=int64))
(array([0, 1], dtype=int64), array([6357, 3643], dtype=int64))
(array([0, 1], dtype=int64), array([57207, 32793], dtype=int64))
(array([0, 1], dtype=int64), array([6357, 3643], dtype=int64))
(array([0, 1], dtype=int64), array([57207, 32793], dtype=int64))
(array([0, 1], dtype=int64), array([6357, 3643], dtype=int64))
(array([0, 1], dtype=int64), array([57208, 32792], dtype=int64))
(array([0, 1], dtype=int64), array([6356, 3644], dtype=int64))
(array([0, 1], dtype=int64), array([57208, 32792], dtype=int64))
(array([0, 1], dtype=int64), array([6356, 3644], dtype=int64))
(array([0, 1], dtype=int64), array([57208, 32792], dtype=int64))
(array([0, 1], dtype=int64), array([6356, 3644], dtype=int64))
(array([0, 1], dtype=int64), array([57208, 32792], dtype=int64))
(array([0, 1], dtype=int64), array([635

In [11]:
test_index

array([   16,    22,    35, ..., 99971, 99990, 99992])

In [21]:
from collections import Counter

In [28]:
word_counter = Counter()

In [24]:
word_counter[0] = 0

In [26]:
word_counter[1] += 1

In [27]:
word_counter

Counter({0: 0, 1: 1})

In [29]:
word_counter[0] += 1

In [30]:
word_counter

Counter({0: 1})

In [31]:
word_counter[1] += 1

In [32]:
word_counter

Counter({0: 1, 1: 1})

In [33]:
id2word = ["aa", "bb"]

In [34]:
reverse = lambda x: dict(zip(x, range(len(x))))

In [35]:
word2id = reverse(id2word)

In [36]:
word2id

{'aa': 0, 'bb': 1}

In [40]:
fold_num = 10
data_file = r"E:/lv_python/NLP/天池大赛_新闻文本分类/tianchidatasets/train_set.csv"
import pandas as pd

In [41]:
import numpy as np

In [43]:
fold_data = []
f = pd.read_csv(data_file, sep='\t', encoding='UTF-8', nrows = 10000)
texts = f['text'].tolist()
labels = f['label'].tolist()

total = len(labels)

index = list(range(total))
np.random.shuffle(index)

In [48]:
all_texts = []
all_labels = []
for i in index:
    all_texts.append(texts[i])
    all_labels.append(labels[i])

label2id = {}
for i in range(total):
    label = str(all_labels[i])
    if label not in label2id:
        label2id[label] = [i]
    else:
        label2id[label].append(i)

In [52]:
a = [[1]]

In [53]:
a.extend([[2]])

In [55]:
import logging

In [56]:
def all_data2fold(fold_num, num=10000):
    fold_data = []
    f = pd.read_csv(data_file, sep='\t', encoding='UTF-8', nrows = num)
    texts = f['text'].tolist()
    labels = f['label'].tolist()

    total = len(labels)

    index = list(range(total))
    np.random.shuffle(index)

    all_texts = []
    all_labels = []
    for i in index:
        all_texts.append(texts[i])
        all_labels.append(labels[i])

    label2id = {}
    for i in range(total):
        label = str(all_labels[i])
        if label not in label2id:
            label2id[label] = [i]
        else:
            label2id[label].append(i)

    all_index = [[] for _ in range(fold_num)]
    for label, data in label2id.items():
        # print(label, len(data))
        batch_size = int(len(data) / fold_num)
        other = len(data) - batch_size * fold_num
        for i in range(fold_num):
            cur_batch_size = batch_size + 1 if i < other else batch_size
            # print(cur_batch_size)
            batch_data = [data[i * batch_size + b] for b in range(cur_batch_size)]
            all_index[i].extend(batch_data)

    batch_size = int(total / fold_num)
    other_texts = []
    other_labels = []
    other_num = 0
    start = 0
    for fold in range(fold_num):
        num = len(all_index[fold])
        texts = [all_texts[i] for i in all_index[fold]]
        labels = [all_labels[i] for i in all_index[fold]]

        if num > batch_size:
            fold_texts = texts[:batch_size]
            other_texts.extend(texts[batch_size:])
            fold_labels = labels[:batch_size]
            other_labels.extend(labels[batch_size:])
            other_num += num - batch_size
        elif num < batch_size:
            end = start + batch_size - num
            fold_texts = texts + other_texts[start: end]
            fold_labels = labels + other_labels[start: end]
            start = end
        else:
            fold_texts = texts
            fold_labels = labels
        assert batch_size == len(fold_labels)

        # shuffle
        index = list(range(batch_size))
        np.random.shuffle(index)

        shuffle_fold_texts = []
        shuffle_fold_labels = []
        for i in index:
            shuffle_fold_texts.append(fold_texts[i])
            shuffle_fold_labels.append(fold_labels[i])

        data = {'label': shuffle_fold_labels, 'text': shuffle_fold_texts}
        fold_data.append(data)

    logging.info("Fold lens %s", str([len(data['label']) for data in fold_data]))

    return fold_data


In [57]:
fold_data = all_data2fold(10)

In [61]:
len(fold_data)

10

In [64]:
fold_data[0]["label"][:2]

[7, 0]

In [63]:
fold_data[0]["text"][:2]

['4462 4381 3578 6909 2522 62 5870 2465 257 5519 4525 1241 4853 5288 5589 1667 1767 4811 2456 4490 6065 3370 332 2456 433 4381 5988 7492 192 1699 5381 4381 3750 4167 5410 6501 299 4381 5988 5381 4381 4429 4646 3370 3370 3370 6242 1291 2538 2506 5445 3530 7058 6045 6569 2420 669 5926 4516 6920 648 4411 4181 4381 6637 5698 2522 5948 62 5870 4462 4381 3578 6909 648 299 2786 3750 669 4490 3750 6175 4381 6637 3945 3272 299 5122 648 4893 4653 980 6017 980 3223 3750 3272 1697 1363 4490 2456 433 4381 5988 803 257 5519 1635 3750 4811 465 3800 7399 4381 6637 299 5122 5977 2109 634 6122 936 900 4167 5410 3750 6501 299 4381 5988 5381 4381 4429 4646 3370 3370 3370 6242 1291 900 4381 6637 7399 3659 3370 3370 2799 1519 2465 4149 2073 4744 4516 1271 1291 2465 4464 5602 4149 1141 2106 648 3495 4220 6515 5920 3750 3618 7399 4490 4392 669 4659 2465 2799 1903 2073 4525 5977 3750 5780 7399 5589 1667 1460 4462 4381 648 3578 6909 1767 5915 2109 868 4516 2990 900 4462 4381 3578 6909 2899 1395 4381 6637 4516 6

In [65]:
# build train, dev, test data
fold_id = 9

# dev
dev_data = fold_data[fold_id]

# train
train_texts = []
train_labels = []
for i in range(0, fold_id):
    data = fold_data[i]
    train_texts.extend(data['text'])
    train_labels.extend(data['label'])

train_data = {'label': train_labels, 'text': train_texts}

In [67]:
train_data["text"][0]

'4462 4381 3578 6909 2522 62 5870 2465 257 5519 4525 1241 4853 5288 5589 1667 1767 4811 2456 4490 6065 3370 332 2456 433 4381 5988 7492 192 1699 5381 4381 3750 4167 5410 6501 299 4381 5988 5381 4381 4429 4646 3370 3370 3370 6242 1291 2538 2506 5445 3530 7058 6045 6569 2420 669 5926 4516 6920 648 4411 4181 4381 6637 5698 2522 5948 62 5870 4462 4381 3578 6909 648 299 2786 3750 669 4490 3750 6175 4381 6637 3945 3272 299 5122 648 4893 4653 980 6017 980 3223 3750 3272 1697 1363 4490 2456 433 4381 5988 803 257 5519 1635 3750 4811 465 3800 7399 4381 6637 299 5122 5977 2109 634 6122 936 900 4167 5410 3750 6501 299 4381 5988 5381 4381 4429 4646 3370 3370 3370 6242 1291 900 4381 6637 7399 3659 3370 3370 2799 1519 2465 4149 2073 4744 4516 1271 1291 2465 4464 5602 4149 1141 2106 648 3495 4220 6515 5920 3750 3618 7399 4490 4392 669 4659 2465 2799 1903 2073 4525 5977 3750 5780 7399 5589 1667 1460 4462 4381 648 3578 6909 1767 5915 2109 868 4516 2990 900 4462 4381 3578 6909 2899 1395 4381 6637 4516 63

In [70]:
class Vocab():
    def __init__(self, train_data):
        self.min_count = 5
        self.pad = 0
        self.unk = 1
        self._id2word = ['[PAD]', '[UNK]']
        self._id2extword = ['[PAD]', '[UNK]']

        self._id2label = []
        self.target_names = []

        self.build_vocab(train_data)

        reverse = lambda x: dict(zip(x, range(len(x))))
        self._word2id = reverse(self._id2word)
        self._label2id = reverse(self._id2label)

        logging.info("Build vocab: words %d, labels %d." % (self.word_size, self.label_size))

    def build_vocab(self, data):
        self.word_counter = Counter()
        
        for text in data['text']:
            words = text.split()
            for word in words:
                self.word_counter[word] += 1

        for word, count in self.word_counter.most_common():
            if count >= self.min_count:
                self._id2word.append(word)

        label2name = {0: '科技', 1: '股票', 2: '体育', 3: '娱乐', 4: '时政', 5: '社会', 6: '教育', 7: '财经',
                      8: '家居', 9: '游戏', 10: '房产', 11: '时尚', 12: '彩票', 13: '星座'}

        self.label_counter = Counter(data['label'])

        for label in range(len(self.label_counter)):
            count = self.label_counter[label]
            self._id2label.append(label)
            self.target_names.append(label2name[label])
        
    def word2id(self, xs):
        if isinstance(xs, list):
            return [self._word2id.get(x, self.unk) for x in xs]
        return self._word2id.get(xs, self.unk)

    def extword2id(self, xs):
        if isinstance(xs, list):
            return [self._extword2id.get(x, self.unk) for x in xs]
        return self._extword2id.get(xs, self.unk)

    def label2id(self, xs):
        if isinstance(xs, list):
            return [self._label2id.get(x, self.unk) for x in xs]
        return self._label2id.get(xs, self.unk)
    
    @property
    def word_size(self):
        return len(self._id2word)

    @property
    def extword_size(self):
        return len(self._id2extword)

    @property
    def label_size(self):
        return len(self._id2label)

In [71]:
vocab = Vocab(train_data)

In [72]:
idx2word = vocab._word2id

In [79]:
data["document"] = data["sentence1"].str.cat(data["sentence2"], sep = ' [SEP] ')

In [84]:
data["document"][0].split(" ")

['1', '2', '3', '4', '5', '6', '7', '[SEP]', '8', '9', '10', '4', '11']

In [102]:
train_data = pd.read_csv("./tianchi_datasets/track3_round1_train.tsv", sep="\t", header=None,
                                 names=["sentence1", "sentence2", "labels"])
train_data["document"] = train_data["sentence1"].str.cat(train_data["sentence2"], sep = " [SEP] ")
test_data = pd.read_csv("./tianchi_datasets/track3_round1_testA.tsv", sep="\t", header=None,
                                names=["sentence1", "sentence2"])
test_data["document"] = test_data["sentence1"].str.cat(test_data["sentence2"], sep=" [SEP] ")

In [103]:
word_counter = Counter()

In [104]:
from tqdm import tqdm

In [105]:
for data in tqdm([train_data, test_data]):
        data["document"] = data["document"].map(lambda x:[x for x in x.split(" ")])

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.24it/s]


In [106]:
train_data = pd.read_csv("./tianchi_datasets/track3_round1_train.tsv", sep="\t", header=None,
                                 names=["sentence1", "sentence2", "labels"])
train_data["document"] = train_data["sentence1"].str.cat(train_data["sentence2"], sep = " [SEP] ")
train_data["document"] = train_data["document"].map(lambda x:[word for word in x.split(" ")])
test_data = pd.read_csv("./tianchi_datasets/track3_round1_testA.tsv", sep="\t", header=None,
                                names=["sentence1", "sentence2"])
test_data["document"] = test_data["sentence1"].str.cat(test_data["sentence2"], sep=" [SEP] ")
test_data["document"] = test_data["document"].map(lambda x: [word for word in x.split(" ")])

In [111]:
def cal_max_length(train_data, test_data):
    # 计算最大输入长度
    max_len = 0
    for data in [train_data, test_data]:
        for x in data["document"]:
            max_len = max(max_len, len(x) + 2) ##这里加2是因为得开头和结尾加上[CLS]和[SEP]

    return max_len

In [112]:
cal_max_length(train_data, test_data)

93

In [113]:
word_counter = Counter()

for data in [train_data, test_data]:
    for text in data["document"]:
         for word in text:
            word_counter[word] += 1

In [127]:
a = word_counter.most_common()

In [123]:
len(word_counter.most_common())

20601

In [151]:
cnt = 5
total_cnt = 0
for x in word_counter.most_common():
    total_cnt += x[1]
    if x[1] < 5:
        cnt += 1 

In [152]:
cnt

13680

In [153]:
total_cnt

1732317

In [128]:
a[14250]

('10733', 1)

In [154]:
vocab = {"[SEP]", "[CLS]"}

In [155]:
type(vocab)

set

In [168]:
min_count = 5
vocab = ['[PAD]', '[SEP]', '[CLS]', '[UNK]']
word_counter = Counter()

for data in [train_data, test_data]:
     for text in data["document"]:
            for word in text:
                 word_counter[word] += 1

for word, count in word_counter.most_common():
    if count >= min_count:
        vocab.append(word)

v2i = {v:i for i,v in enumerate(vocab)}
i2v = {i:v for v, i in v2i.items()}

In [165]:
{'[PAD]', '[SEP]', '[CLS]', '[UNK]'}

{'[CLS]', '[PAD]', '[SEP]', '[UNK]'}

In [169]:
len(vocab)

6930

In [161]:
len(word_counter)

20601

In [172]:
cnt = 0
for word, count in word_counter.most_common():
    if count >= 2:
        cnt += 1

In [173]:
cnt

12364

In [179]:
vocab = ['[PAD]', '[SEP]', '[CLS]', '[UNK]']
word_counter = Counter()

for data in [train_data, test_data]:
    for text in data["document"]:
        for word in text:
            word_counter[word] += 1
        
for word, count in word_counter.most_common():
        vocab.append(word)

v2i = {v:i for i,v in enumerate(vocab)}
i2v = {i:v for v, i in v2i.items()}

In [183]:
vocab = ['[PAD]', '[CLS]', '[UNK]']
word_counter = Counter()

for data in [train_data, test_data]:
    for text in data["document"]:
        for word in text:
            word_counter[word] += 1
        
for word, count in word_counter.most_common():
    vocab.append(word)

    

In [186]:
v2i = {v:i for i,v in enumerate(vocab)}
i2v = {i:v for v, i in v2i.items()}

In [188]:
len(v2i)

20604

In [190]:
v2i["21317"]

20019

In [261]:
def get_vocab_v2i_i2v(train_data, test_data):
        ## 获得语料库vocab, id2word, word2id
        #min_count = 5
        vocab = ['[PAD]', '[UNK]', '[CLS]']
        word_counter = Counter()

        for data in [train_data, test_data]:
            for text in data["document"]:
                for word in text:
                    word_counter[word] += 1

        for word, count in word_counter.most_common():
            vocab.append(word)

        v2i = {v:i for i,v in enumerate(vocab)}
        i2v = {i:v for v, i in v2i.items()}

        return vocab, v2i, i2v

In [262]:
vocab, v2i, i2v = get_vocab_v2i_i2v(train_data, test_data)

In [266]:
v2i

{'[PAD]': 0,
 '[UNK]': 1,
 '[CLS]': 2,
 '[SEP]': 3,
 '12': 4,
 '29': 5,
 '19': 6,
 '23': 7,
 '16': 8,
 '11': 9,
 '10': 10,
 '9': 11,
 '32': 12,
 '39': 13,
 '161': 14,
 '48': 15,
 '126': 16,
 '243': 17,
 '43': 18,
 '106': 19,
 '5': 20,
 '6': 21,
 '8': 22,
 '72': 23,
 '13': 24,
 '283': 25,
 '66': 26,
 '282': 27,
 '370': 28,
 '459': 29,
 '28': 30,
 '317': 31,
 '62': 32,
 '247': 33,
 '76': 34,
 '140': 35,
 '50': 36,
 '46': 37,
 '276': 38,
 '169': 39,
 '80': 40,
 '47': 41,
 '59': 42,
 '202': 43,
 '300': 44,
 '217': 45,
 '45': 46,
 '127': 47,
 '134': 48,
 '426': 49,
 '431': 50,
 '536': 51,
 '67': 52,
 '230': 53,
 '447': 54,
 '133': 55,
 '24': 56,
 '52': 57,
 '522': 58,
 '533': 59,
 '227': 60,
 '251': 61,
 '432': 62,
 '226': 63,
 '253': 64,
 '453': 65,
 '168': 66,
 '358': 67,
 '25': 68,
 '629': 69,
 '176': 70,
 '272': 71,
 '267': 72,
 '462': 73,
 '55': 74,
 '102': 75,
 '153': 76,
 '79': 77,
 '421': 78,
 '173': 79,
 '130': 80,
 '334': 81,
 '524': 82,
 '101': 83,
 '347': 84,
 '335': 85,
 '525':

In [216]:
test_size = 0.1
seed_val = 2021

In [253]:
X_train, X_val, y_train, y_val = train_test_split(
            train_data["document"], train_data["labels"], test_size=test_size,
            stratify=train_data["labels"], random_state=seed_val)

In [271]:
def convert_to_bert_datasets(data, labels, max_length, v2i):
    input_ids = []

    for inputs in data:
        padded = np.full(max_length, fill_value=0, dtype=np.int32)
        padded[:(len(inputs) + 2)] = [v2i['[CLS]']] + [v2i.get(x, 1) for x in inputs] + [v2i['[SEP]']]
        input_ids.append(torch.LongTensor([padded]))

    input_ids = torch.cat(input_ids, dim=0)

    if labels is None:
        return Data.TensorDataset(input_ids)
    else:
        labels = torch.tensor(labels.values)
        return Data.TensorDataset(input_ids, labels)

In [272]:
train = convert_to_bert_datasets(X_train, y_train, max_len, v2i)

In [273]:
train_dataloader = Data.DataLoader(train, batch_size=batch_size, shuffle=False)

In [274]:
for x in train_dataloader:
    print(x)
    break

[tensor([[  2,  23,   5,  ...,   0,   0,   0],
        [  2, 236, 283,  ...,   0,   0,   0],
        [  2, 151, 617,  ...,   0,   0,   0],
        ...,
        [  2,   4, 198,  ...,   0,   0,   0],
        [  2,  27,  25,  ...,   0,   0,   0],
        [  2,   7,   6,  ...,   0,   0,   0]]), tensor([0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1])]


In [255]:
v2i.get("72")

23

In [275]:
x[:-1][0][0]

tensor([  2,  23,   5, 138, 410, 146,   3,  10, 792,  18, 410, 146,  23,   5,
        138,  20,  21,  45,  16,   3,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0])

In [276]:
X_train[38335]

['72',
 '29',
 '4',
 '290',
 '33',
 '[SEP]',
 '10',
 '772',
 '43',
 '290',
 '33',
 '72',
 '29',
 '4',
 '5',
 '6',
 '217',
 '126']

In [279]:
from Data_generator_modified import Corpus

In [280]:
corpus = Corpus( batch_size , seed_val)
train_loader, valid_loader, test_loader = corpus.get_loaders()

********************************Data_precessing*********************************
## dataset size is 100000
*******************************Load  dataset ....*******************************
## Load Datasets Consume 0:00:06 s ###
************************All Train and Test Data loaded !************************


In [282]:
for x in test_loader:
    print(x)
    break

[tensor([[  1,   4,   7,  ...,   0,   0,   0],
        [  1,   4,  20,  ...,   0,   0,   0],
        [  1,  48, 573,  ...,   0,   0,   0],
        ...,
        [  1, 953,   5,  ...,   0,   0,   0],
        [  1, 262,   9,  ...,   0,   0,   0],
        [  1,   5,  32,  ...,   0,   0,   0]])]


In [284]:
from torch import nn

In [295]:
emb = nn.Embedding(16, 3)
dropout = nn.Dropout(0.5)

In [290]:
emb.state_dict()

OrderedDict([('weight',
              tensor([[-5.4280e-01,  6.7888e-01, -4.2775e-01],
                      [ 5.4533e-01,  9.4880e-02,  2.6756e-01],
                      [-1.2994e+00,  9.4631e-04,  1.8252e-01],
                      [ 2.2095e+00, -6.1967e-01,  2.6030e-01],
                      [ 3.7188e-01, -5.8912e-01, -1.1403e-01],
                      [ 2.3440e+00,  6.3106e-01, -5.6090e-01],
                      [ 1.5914e+00,  2.0060e+00, -3.5820e-01],
                      [-6.9950e-01, -9.5084e-02, -1.6080e+00],
                      [ 1.4059e+00,  1.0950e+00, -1.6617e+00],
                      [-1.0668e+00, -1.2655e+00, -1.0872e-01],
                      [-4.1067e-01, -7.9915e-01, -1.0813e+00],
                      [ 3.5618e-01, -6.2622e-01,  7.8715e-01],
                      [-2.5217e-01,  4.2111e-01,  6.4201e-01],
                      [-1.0840e+00, -4.7387e-01,  3.8295e-01],
                      [ 1.2236e-01,  6.9582e-01, -1.5468e+00],
                      [-5.2373e

In [296]:
a = torch.rand(10, 2)

In [299]:
emb = nn.Embedding(3, 5)

In [300]:
inputsa = torch.LongTensor([1, 2, 0])

In [302]:
emb(inputsa)

tensor([[-0.0885,  0.6042,  1.5657,  0.5765, -2.6056],
        [ 0.0842,  0.2499,  0.8024,  1.9202, -1.2176],
        [ 0.5256,  0.7225,  0.4041, -0.5163,  0.1360]],
       grad_fn=<EmbeddingBackward>)

In [ ]:
import pandas as pd
train_data = 

In [95]:
import json
import pandas as pd



def create_new_traindata(test_score, train_data, test_data):
    tmp = []
    dir_path = os.getcwd()
    
    with open(os.path.join(dir_path, test_score) , "r", encoding="utf-8") as f:
        for line in f:
            content = json.loads(line)
            score = content["label"]
            if score > 0.95:
                tmp.append(1)
            elif score < 0.05:
                tmp.append(0)
            else:
                tmp.append("unk")
    
    with open(os.path.join(dir_path, test_data), "r", encoding="utf-8") as fr:
        with open("tianchi_datasets/track3_round1_testA_label.tsv", "w", encoding="utf-8") as fw:
            data = fr.readlines()
            for i in range(len(data)):
                content = data[i].strip()
                label = tmp[i]
                if label in [0, 1]:
                    lis = content + "\t" + str(label) + "\n"
                else:
                    lis = ""
                fw.write(lis)
    
    train_data = pd.read_csv(os.path.join(dir_path, train_data), sep="\t", header=None,
                                 names=["sentence1", "sentence2", "labels"])
    test_data = pd.read_csv("./tianchi_datasets/track3_round1_testA_label.tsv", sep="\t", header=None,
                                 names=["sentence1", "sentence2", "labels"])
    
    new_data = pd.concat([train_data, test_data], axis = 0)
    new_data.to_csv("./tianchi_datasets/track3_round1_newtrain.tsv", sep = "\t", header=None, index = None)

In [96]:
import os
os.getcwd()

'E:\\lv_python\\NLP\\阿里天池小布助手'

In [97]:
test_score = "tianchi_datasets/test.json"
train_data = "tianchi_datasets/track3_round1_train.tsv"
test_data = "tianchi_datasets/track3_round1_testA.tsv"

In [98]:
create_new_traindata(test_score, train_data, test_data)

In [33]:
len(data)

25000

In [43]:
import pandas as pd

In [44]:
train_data = pd.read_csv("./tianchi_datasets/track3_round1_train.tsv", sep="\t", header=None,
                                 names=["sentence1", "sentence2", "labels"])

In [45]:
train_data

,sentence1,sentence2,labels
0,1 2 3 4 5 6 7,8 9 10 4 11,0
1,12 13 14 15,12 15 11 16,0
2,17 18 12 19 20 21 22 23 24,12 23 25 6 26 27 19,1
3,28 29 30 31 11,32 33 34 30 31,1
4,29 35 36 29,29 37 36 29,1
...,...,...,...
99995,12 19 1162 126 53 66,12 19 79 389 126 53 66,1
99996,275 552 553 433 881 338 1104 101 202 2343 14825,995 551 550 1660 2830 1075 662 935,0
99997,421 330 62 12 80 81 82 76,202 62 12 80 838 76,1
99998,177 455 456 3474 964 1364 55 1364,133 134 2246,1


In [46]:
new_data = pd.read_csv("./tianchi_datasets/track3_round1_train_new.tsv", sep="\t", header=None,
                                 names=["sentence1", "sentence2", "labels"])

In [50]:
new_data1 = pd.concat([train_data, new_data], axis = 0)

In [52]:
new_data1.to_csv("./tianchi_datasets/track3_round1_train_new1.tsv", sep = "\t", header=None, index = None)

In [53]:
new_data2 = pd.read_csv("./tianchi_datasets/track3_round1_train_new1.tsv", sep="\t", header=None,
                                 names=["sentence1", "sentence2", "labels"])

In [54]:
new_data2

,sentence1,sentence2,labels
0,1 2 3 4 5 6 7,8 9 10 4 11,0
1,12 13 14 15,12 15 11 16,0
2,17 18 12 19 20 21 22 23 24,12 23 25 6 26 27 19,1
3,28 29 30 31 11,32 33 34 30 31,1
4,29 35 36 29,29 37 36 29,1
...,...,...,...
115401,29 243 2107 2109 208,29 275 272 91 2108,1
115402,4915 23 39 9 3544 19,158 478 426 19 1274 120 212 1378 157,0
115403,29 81 140 72 12 5 1763 1764 12 459 16,1553 72 12 20811 6722 16,1
115404,740 2155 6667,20273 762 852 11,0


In [63]:
train_data["document"] = train_data["sentence1"].str.cat(train_data["sentence2"], sep = " [SEP] ")

In [76]:
inputs = train_data["document"][99999]

In [75]:
train_data["document"][99999]

'29 168 12 19 1003 719 23 29 263 276 [SEP] 29 23 12 115 115 263 16'

In [68]:
inputs

'1 2 3 4 5 6 7 [SEP] 8 9 10 4 11'

In [65]:
from transformers import BertTokenizer

In [66]:
tokenizer = BertTokenizer.from_pretrained("./pretrain_model/chinese_roberta_wwm_ext/")

In [77]:
input_ids = tokenizer(inputs)

In [78]:
input_ids

{'input_ids': [101, 8162, 8832, 8110, 8131, 8135, 8152, 8459, 8160, 8133, 8162, 10864, 11173, 102, 8162, 8133, 8110, 8787, 8787, 10864, 8121, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [79]:
tokenizer.convert_ids_to_tokens(input_ids["input_ids"])

['[CLS]',
 '29',
 '168',
 '12',
 '19',
 '100',
 '##3',
 '71',
 '##9',
 '23',
 '29',
 '263',
 '276',
 '[SEP]',
 '29',
 '23',
 '12',
 '115',
 '115',
 '263',
 '16',
 '[SEP]']

[0,
 'unk',
 0,
 0,
 0,
 'unk',
 0,
 'unk',
 0,
 'unk',
 'unk',
 0,
 1,
 0,
 'unk',
 'unk',
 0,
 0,
 'unk',
 'unk',
 'unk',
 1,
 0,
 1,
 'unk',
 0,
 'unk',
 0,
 0,
 0,
 1,
 'unk',
 0,
 'unk',
 1,
 'unk',
 0,
 0,
 'unk',
 'unk',
 0,
 'unk',
 1,
 0,
 1,
 1,
 'unk',
 0,
 'unk',
 'unk',
 0,
 1,
 'unk',
 0,
 'unk',
 0,
 1,
 'unk',
 'unk',
 'unk',
 0,
 'unk',
 0,
 'unk',
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 'unk',
 0,
 0,
 0,
 'unk',
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 'unk',
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 'unk',
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 'unk',
 'unk',
 'unk',
 0,
 1,
 1,
 'unk',
 'unk',
 1,
 'unk',
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 'unk',
 1,
 1,
 1,
 0,
 'unk',
 'unk',
 'unk',
 0,
 'unk',
 0,
 0,
 0,
 'unk',
 1,
 0,
 0,
 1,
 'unk',
 1,
 'unk',
 0,
 'unk',
 'unk',
 'unk',
 'unk',
 1,
 1,
 'unk',
 'unk',
 'unk',
 0,
 'unk',
 1,
 'unk',
 'unk',
 1,
 1,
 1,
 'unk',
 'unk',
 1,
 'unk',
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 'unk',
 0,
 'unk',
 1,
 0,
 'unk',
 'unk',
 1,
 0,
 'un

In [ ]:
ocnli_train = pd.read_csv("./tianchi_datasets/track3_round1_train_new.tsv", sep="\t", header=None,
                                  quoting=3, encoding="utf-8", names=["id", "sentence1", "sentence2", "class"])
        ocnli_train["document"] = ocnli_train["sentence1"].str.cat(ocnli_train["sentence2"], sep=" [SEP] ")
        ocnli_test = pd.read_csv("./tianchi_datasets/track3_round1_testA.tsv", sep="\t", header=None,
                                 quoting=3, encoding="utf-8", names=["id", "sentence1", "sentence2"])
        ocnli_test["document"] = ocnli_test["sentence1"].str.cat(ocnli_test["sentence2"], sep=" [SEP] ")

In [104]:
train_data

'tianchi_datasets/track3_round1_train.tsv'

In [118]:
dir_path = os.getcwd()
train = pd.read_csv(os.path.join(dir_path, train_data), sep="\t", header=None,
                                 names=["sentence1", "sentence2", "labels"])
test = pd.read_csv("./tianchi_datasets/track3_round1_testA.tsv", sep="\t", header=None,
                                 names=["sentence1", "sentence2", "labels"])

In [121]:
train["sentence1"].str.len().describe(percentiles = [.99, .999, .9999]), train["sentence2"].str.len().describe(percentiles = [.99, .999, .9999])\
,test["sentence1"].str.len().describe(percentiles = [.99, .999, .9999]), test["sentence2"].str.len().describe(percentiles = [.99, .999, .9999])

(count     100000.000000
 mean          23.421830
 std           10.443222
 min            3.000000
 50%           21.000000
 99%           56.000000
 99.9%         90.000000
 99.99%       159.000600
 max          236.000000
 Name: sentence1, dtype: float64,
 count     100000.000000
 mean          23.410970
 std           10.431748
 min            3.000000
 50%           21.000000
 99%           56.000000
 99.9%         91.000000
 99.99%       144.000100
 max          185.000000
 Name: sentence2, dtype: float64,
 count     25000.000000
 mean         22.314120
 std           9.104125
 min           4.000000
 50%          21.000000
 99%          50.000000
 99.9%        63.000000
 99.99%       91.000000
 max         225.000000
 Name: sentence1, dtype: float64,
 count     25000.000000
 mean         22.388520
 std           9.290472
 min           4.000000
 50%          21.000000
 99%          50.000000
 99.9%        68.000000
 99.99%      101.501300
 max         311.000000
 Name: sentence2

In [127]:
import torch
import numpy as np

In [135]:
weight=torch.from_numpy(np.array([0.1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])).float()

In [136]:
weight1 = torch.tensor([0.1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])

In [137]:
weight == weight

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)

In [138]:
import numpy as np
from collections import Counter
import itertools

In [139]:
docs = [
    "it is a good day, I like to stay here",
    "I am happy to be here",
    "I am bob",
    "it is sunny today",
    "I have a party today",
    "it is a dog and that is a cat",
    "there are dog and cat on the tree",
    "I study hard this morning",
    "today is a good day",
    "tomorrow will be a good day",
    "I like coffee, I like book and I like apple",
    "I do not like it",
    "I am kitty, I like bob",
    "I do not care who like bob, but I like kitty",
    "It is coffee time, bring your cup",
]

In [140]:
docs_words = [d.replace(",", "").split(" ") for d in docs]

In [143]:
vocab = set(itertools.chain(*docs_words))

In [145]:
v2i = {v: i for i, v in enumerate(vocab)}
i2v = {i: v for v, i in v2i.items()}

In [146]:
tf_methods = {
        "log": lambda x: np.log(1+x),
        "augmented": lambda x: 0.5 + 0.5 * x / np.max(x, axis=1, keepdims=True),
        "boolean": lambda x: np.minimum(x, 1),
        "log_avg": lambda x: (1 + safe_log(x)) / (1 + safe_log(np.mean(x, axis=1, keepdims=True))),
    }
idf_methods = {
        "log": lambda x: 1 + np.log(len(docs) / (x+1)),
        "prob": lambda x: np.maximum(0, np.log((len(docs) - x) / (x+1))),
        "len_norm": lambda x: x / (np.sum(np.square(x))+1),
    }

In [147]:
_tf = np.zeros((len(vocab), len(docs)), dtype=np.float64)

In [149]:
i = 0
d = docs_words[0]

In [152]:
counter = Counter(d)

In [166]:
tf_methods.get("log")(2)

1.0986122886681098

In [167]:
i2v

{0: 'there',
 1: 'cat',
 2: 'sunny',
 3: 'happy',
 4: 'good',
 5: 'apple',
 6: 'have',
 7: 'care',
 8: 'your',
 9: 'bring',
 10: 'am',
 11: 'day',
 12: 'the',
 13: 'it',
 14: 'who',
 15: 'is',
 16: 'this',
 17: 'like',
 18: 'a',
 19: 'study',
 20: 'not',
 21: 'tree',
 22: 'coffee',
 23: 'be',
 24: 'that',
 25: 'party',
 26: 'do',
 27: 'I',
 28: 'today',
 29: 'to',
 30: 'kitty',
 31: 'will',
 32: 'cup',
 33: 'morning',
 34: 'and',
 35: 'stay',
 36: 'are',
 37: 'but',
 38: 'hard',
 39: 'dog',
 40: 'It',
 41: 'time',
 42: 'tomorrow',
 43: 'bob',
 44: 'on',
 45: 'here',
 46: 'book'}

In [160]:
counter.most_common(1)

[('it', 1)]

In [168]:
df = np.zeros((len(i2v), 1))
for i in range(len(i2v)):
        d_count = 0
        for d in docs_words:
            d_count += 1 if i2v[i] in d else 0
        df[i, 0] = d_count

In [174]:
def get_tf(method="log"):
    # term frequency: how frequent a word appears in a doc
    _tf = np.zeros((len(vocab), len(docs)), dtype=np.float64)    # [n_vocab, n_doc]
    for i, d in enumerate(docs_words):
        counter = Counter(d)
        for v in counter.keys():
            _tf[v2i[v], i] = counter[v] / counter.most_common(1)[0][1]

    weighted_tf = tf_methods.get(method, None)
    if weighted_tf is None:
        raise ValueError
    return weighted_tf(_tf)


def get_idf(method="log"):
    # inverse document frequency: low idf for a word appears in more docs, mean less important
    df = np.zeros((len(i2v), 1))
    for i in range(len(i2v)):
        d_count = 0
        for d in docs_words:
            d_count += 1 if i2v[i] in d else 0
        df[i, 0] = d_count

        
    idf_fn = idf_methods.get(method, None)
    if idf_fn is None:
        raise ValueError
    return idf_fn(df)

In [175]:
tf = get_tf()           # [n_vocab, n_doc]
idf = get_idf()         # [n_vocab, 1]
tf_idf = tf * idf       # [n_vocab, n_doc]
print("tf shape(vocab in each docs): ", tf.shape)
print("\ntf samples:\n", tf[:2])
print("\nidf shape(vecb in all docs): ", idf.shape)
print("\nidf samples:\n", idf[:2])
print("\ntf_idf shape: ", tf_idf.shape)
print("\ntf_idf sample:\n", tf_idf[:2])

tf shape(vocab in each docs):  (47, 15)

tf samples:
 [[0.         0.         0.         0.         0.         0.
  0.69314718 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.40546511
  0.69314718 0.         0.         0.         0.         0.
  0.         0.         0.        ]]

idf shape(vecb in all docs):  (47, 1)

idf samples:
 [[3.01490302]
 [2.60943791]]

tf_idf shape:  (47, 15)

tf_idf sample:
 [[0.         0.         0.         0.         0.         0.
  2.08977153 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.         1.05803603
  1.80872453 0.         0.         0.         0.         0.
  0.         0.         0.        ]]


In [182]:
len(tf[0])

15

In [179]:
tf[:, 0]

array([0.        , 0.        , 0.        , 0.        , 0.69314718,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.69314718, 0.        , 0.69314718, 0.        ,
       0.69314718, 0.        , 0.69314718, 0.69314718, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.69314718, 0.        , 0.69314718,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.69314718, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.69314718, 0.        ])

In [177]:
tf[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.69314718, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [183]:
col = tf_idf[:, 0]

In [184]:
col

array([0.        , 0.        , 0.        , 0.        , 1.60931851,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 1.60931851, 0.        , 1.45464719, 0.        ,
       1.32827152, 0.        , 1.32827152, 1.32827152, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.97419418, 0.        , 1.80872453,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       2.08977153, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       1.80872453, 0.        ])

In [186]:
col[35]

2.0897715283505334

In [185]:
np.argsort(col)

array([ 0, 24, 25, 26, 28, 30, 31, 32, 33, 22, 34, 37, 38, 39, 40, 41, 42,
       43, 44, 36, 21, 23, 19,  1,  2,  3,  5,  6,  7,  8,  9, 20, 10, 14,
       16, 12, 46, 27, 15, 17, 18, 13, 11,  4, 29, 45, 35], dtype=int64)

In [187]:
a = np.array([[2, 2], [1, 1]])

In [196]:
tf[1]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.40546511, 0.69314718, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [197]:
tf_idf[1]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       1.05803603, 1.80872453, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [199]:
idf[1]

array([2.60943791])

In [190]:
b = np.array([[2],[2]])

In [191]:
b

array([[2],
       [2]])

In [192]:
a * b

array([[4, 4],
       [2, 2]])

In [201]:
q = "I get a coffee cup"

In [202]:
np.sum([1, 2, 3], keepdims = True)

array([6])

In [218]:
idf.T.dot(tf).ravel() == idf.T.dot(tf).flatten()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [ ]:
docs_words = [d.replace(",", "").split(" ") for d in docs]
vocab = set(itertools.chain(*docs_words))
v2i = {v: i for i, v in enumerate(vocab)}
i2v = {i: v for v, i in v2i.items()}

In [222]:
train_data = pd.read_csv("./tianchi_datasets/track3_round1_train.tsv")
test_data = pd.read_csv("./tianchi_datasets/track3_round1_testA.tsv")

In [ ]:
train_data["document"] = train_data[""]

In [541]:
train_data = pd.read_csv("./tianchi_datasets/track3_round1_train.tsv", sep="\t", header=None,nrows = 10000,
                                  quoting=3, encoding="utf-8", names=["sentence1", "sentence2", "class"])
train_data["document"] = train_data["sentence1"].str.cat(train_data["sentence2"], sep=" ")
test_data = pd.read_csv("./tianchi_datasets/track3_round1_testA.tsv", sep="\t", header=None, nrows = 100,
                                 quoting=3, encoding="utf-8", names=["sentence1", "sentence2"])
test_data["document"] = test_data["sentence1"].str.cat(test_data["sentence2"], sep=" ")

In [542]:
docs = []
q = []
for word1, word2 in itertools.zip_longest(train_data["document"], test_data["document"]):
    docs.append(word1)
    if word2 is None:
        continue
    q.append(word2)
    

In [543]:
docs_words = [d.replace(",", "").split(" ") for d in docs]
vocab = set(itertools.chain(*docs_words))
v2i = {v: i for i, v in enumerate(vocab)}
i2v = {i: v for v, i in v2i.items()}

In [544]:
tf_methods = {
        "log": lambda x: np.log(1+x),
        "augmented": lambda x: 0.5 + 0.5 * x / np.max(x, axis=1, keepdims=True),
        "boolean": lambda x: np.minimum(x, 1),
        "log_avg": lambda x: (1 + safe_log(x)) / (1 + safe_log(np.mean(x, axis=1, keepdims=True))),
    }
idf_methods = {
        "log": lambda x: 1 + np.log(len(docs) / (x+1)),
        "prob": lambda x: np.maximum(0, np.log((len(docs) - x) / (x+1))),
        "len_norm": lambda x: x / (np.sum(np.square(x))+1),
    }

In [545]:
def get_tf(method="log"):
    # term frequency: how frequent a word appears in a doc
    _tf = np.zeros((len(vocab), len(docs)), dtype=np.float64)    # [n_vocab, n_doc]
    for i, d in enumerate(docs_words):
        counter = Counter(d)
        for v in counter.keys():
            _tf[v2i[v], i] = counter[v] / counter.most_common(1)[0][1]

    weighted_tf = tf_methods.get(method, None)
    if weighted_tf is None:
        raise ValueError
    return weighted_tf(_tf)

In [546]:
def get_idf(method="log"):
    # inverse document frequency: low idf for a word appears in more docs, mean less important
    df = np.zeros((len(i2v), 1))
    for i in range(len(i2v)):
        d_count = 0
        for d in docs_words:
            d_count += 1 if i2v[i] in d else 0
        df[i, 0] = d_count

    idf_fn = idf_methods.get(method, None)
    if idf_fn is None:
        raise ValueError
    return idf_fn(df)

In [547]:
def cosine_similarity(q, _tf_idf):
    unit_q = q / np.sqrt(np.sum(np.square(q), axis=0, keepdims=True))
    unit_ds = _tf_idf / np.sqrt(np.sum(np.square(_tf_idf), axis=0, keepdims=True))
    similarity = unit_ds.T.dot(unit_q).ravel()
    return similarity

In [548]:
def docs_score(q, len_norm=False):
    global idf, tf_idf
    q_words = q.replace(",", "").split(" ")

    # add unknown words
    unknown_v = 0
    for v in set(q_words):
        if v not in v2i:
            v2i[v] = len(v2i)
            i2v[len(v2i)-1] = v
            unknown_v += 1
    if unknown_v > 0:
        _idf = np.concatenate((idf, np.zeros((unknown_v, 1), dtype=np.float)), axis=0)
        _tf_idf = np.concatenate((tf_idf, np.zeros((unknown_v, tf_idf.shape[1]), dtype=np.float)), axis=0)
    else:
        _idf, _tf_idf = idf, tf_idf
    counter = Counter(q_words)
    q_tf = np.zeros((len(_idf), 1), dtype=np.float)     # [n_vocab, 1]
    for v in counter.keys():
        q_tf[v2i[v], 0] = counter[v]

    q_vec = q_tf * _idf            # [n_vocab, 1]

    q_scores = cosine_similarity(q_vec, _tf_idf)
    if len_norm:
        len_docs = [len(d) for d in docs_words]
        q_scores = q_scores / np.array(len_docs)
    
    idf, tf_idf = _idf, _tf_idf
    return q_scores

In [549]:
tf = get_tf()           # [n_vocab, n_doc]
idf = get_idf()         # [n_vocab, 1]
tf_idf = tf * idf       # [n_vocab, n_doc]

In [524]:
tmp = []
for content in tqdm(q):
    scores = docs_score(content)
    score = max(scores)
    idx = np.argmax(scores)
    if score > 0.7:
        label = train_data["class"][idx]
    else:
        label = 2
    tmp.append(label)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 29.83it/s]


In [526]:
test_data["label"] = tmp
newtest_1 = test_data[test_data["label"] < 2]
newtest = pd.read_csv("./tianchi_datasets/track3_round1_testA_label.tsv", sep = "\t", 
                     header = None, names = ["sentence1", "sentence2", "label"])
newtest2 = pd.merge(newtest_1, newtest,  how = "inner")
newtrain = pd.concat([train_data, newtest2])

In [ ]:
newtrain.to_csv("./tianchi_datasets/track3_round1_newtrain.tsv", sep="\t", header=None, index=None)

NameError: name 'newtest1' is not defined

In [557]:
train_data = pd.read_csv("./tianchi_datasets/track3_round1_train.tsv", sep="\t", header=None,
                                  quoting=3, encoding="utf-8", names=["sentence1", "sentence2", "label"])
train_data["document"] = train_data["sentence1"].str.cat(train_data["sentence2"], sep=" [SEP] ")
test_data = pd.read_csv("./tianchi_datasets/track3_round1_testA.tsv", sep="\t", header=None,
                                 quoting=3, encoding="utf-8", names=["sentence1", "sentence2"])
test_data["document"] = test_data["sentence1"].str.cat(test_data["sentence2"], sep=" [SEP] ")

In [559]:
from transformers import BertTokenizer

In [563]:
tokenizer = BertTokenizer.from_pretrained("./pretrain_model/chinese_roberta_wwm_ext")

In [566]:
def cal_max_length(data):
    # 计算最大输入长度
    max_len = 0
    for x in tqdm(data["document"]):
        max_len = max(max_len, len(tokenizer(x)["input_ids"]))
    return max_len

In [567]:
cal_max_length(train_data)

100%|████████████████████████████████████████████████████████████████████████| 100000/100000 [00:51<00:00, 1948.89it/s]


97

In [570]:
test_data = pd.read_csv("./tianchi_datasets/track3_round1_testA.tsv", sep="\t", header=None,
                                 quoting=3, encoding="utf-8", names=["sentence1", "sentence2"])
test_data["document"] = test_data["sentence1"].str.cat(test_data["sentence2"], sep=" [SEP] ")

In [571]:
cal_max_length(test_data)

100%|██████████████████████████████████████████████████████████████████████████| 25000/25000 [00:11<00:00, 2142.60it/s]


152

In [573]:
tokenizer(train_data["document"][0])

{'input_ids': [101, 122, 123, 124, 125, 126, 127, 128, 102, 129, 130, 8108, 125, 8111, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [583]:

import torch.utils.data as Data
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')


def convert_to_bert_dataset(data, labels, tokenizer, max_length, name):
    input_ids = []
    token_type_ids = []
    attention_masks = []

    for x in tqdm(data, desc="convert_to_bert_{}_dataset".format(name)):
        encoded_input = tokenizer.encode_plus(x, add_special_tokens=True,
                                              max_length=max_length, padding="max_length",
                                              return_attention_mask=True,
                                              return_tensors="pt", truncation=True)

        input_ids.append(encoded_input["input_ids"])
        attention_masks.append(encoded_input["attention_mask"])
        # 对于bert,输入有token_type_ids,其他模型没有
        try:
            token_type_ids.append(encoded_input["token_type_ids"])
        except:
            pass

    # convert lists to tensor, bert在Pytorch中只接受torch格式的输入
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    if len(token_type_ids) != 0:
        token_type_ids = torch.cat(token_type_ids, dim=0)

    # 对于测试集，没有labels
    if labels is None:
        # 对于非bert类输入,没有token_type_ids
        if len(token_type_ids) == 0:
            return Data.TensorDataset(input_ids, attention_masks)
        return Data.TensorDataset(input_ids, attention_masks, token_type_ids)

    else:
        labels = torch.tensor(labels.values)
        if len(token_type_ids) == 0:
            return Data.TensorDataset(input_ids, attention_masks, labels)
        return Data.TensorDataset(input_ids, attention_masks, token_type_ids, labels)




class Corpus():

    def __init__(self, tokenizer, batch_size, seed_val):

        self.batch_size = batch_size
        self.seed_val = seed_val
        # 保持验证集和测试集数量大致相同,约为1500,1500/53386 = 0.0281
        # 0.1表示10折交叉验证,训练集和验证集比例
        self.test_scale = 0.1
        self.tokenizer = tokenizer

        print(f"{'Data_precessing':*^80}")
        """
        “当文本文件中带有英文双引号时，直接用pd.read_csv进行读取会导致行数减少，
        此时应该对read_csv设置参数quoting=3或者quoting=csv.QUOTE_NONE”
        不设置quoting，默认会去除英文双引号，只留下英文双引号内的内容，设置quoting = 3，会如实读取内容。
        """
        train_data = pd.read_csv("./tianchi_datasets/track3_round1_newtrain.tsv", sep="\t", header=None,
                                  quoting=3, encoding="utf-8", names=["sentence1", "sentence2", "labels"])
        train_data["document"] = train_data["sentence1"].str.cat(train_data["sentence2"], sep=" [SEP] ")
        test_data = pd.read_csv("./tianchi_datasets/track3_round1_testA.tsv", sep="\t", header=None,
                                 quoting=3, encoding="utf-8", names=["sentence1", "sentence2"])
        test_data["document"] = test_data["sentence1"].str.cat(test_data["sentence2"], sep=" [SEP] ")

        print("## dataset size is {}".format(len(train_data)))

        # 计算max length, 计算耗时,直接保存结果，大概需要耗时30s
        self.max_length = 152
        """
        max_len = max(self.cal_max_length(ocnli_train), self.cal_max_length(ocnli_test))
        print("!!!### max token length of FineTune model : ", max_len)
        """

        print(f"{'Load OCNLI dataset ....':*^80}")
        self.train_loader, self.valid_loader, self.test_loader,\
            = self.load_generator(self.test_scale,
            train_data[["document", "labels"]], test_data[["document"]], self.batch_size)

        print(f"{'All Train and Test Data loaded !':*^80}")

    def load_generator(self, test_size, train_data, test_data, batch_size):
        """生成训练、验证、测试集合的dataloader"""


        # 划分训练集和验证集,同时根据label值分层抽样
        X_train, X_val, y_train, y_val = train_test_split(
            train_data["document"], train_data["labels"], test_size=test_size,
            stratify= train_data["labels"], random_state=self.seed_val)

        train = convert_to_bert_dataset(X_train, y_train, self.tokenizer, self.max_length, "train")
        validation = convert_to_bert_dataset(X_val, y_val, self.tokenizer, self.max_length, "valid")
        test = convert_to_bert_dataset(test_data["document"], None, self.tokenizer, self.max_length, "test")

        train_dataloader = Data.DataLoader(
            train, batch_size=batch_size, shuffle=True)
        valid_dataloader = Data.DataLoader(
            validation, batch_size=batch_size, shuffle=False)
        ##因为要逐条预测
        test_dataloader = Data.DataLoader(test, batch_size=1, shuffle=False)

        return train_dataloader, valid_dataloader, test_dataloader



    def cal_max_length(self, data):
        # 计算最大输入长度
        max_len = 0
        for x in tqdm(data["document"]):
            max_len = max(max_len, len(self.tokenizer(x)["input_ids"]))
        return max_len


    def get_loaders(self):
        return self.train_loader, self.valid_loader, self.test_loader

In [423]:
test_data["label"] = list(range(100))

In [575]:
batch_size = 4
seed_val = 4

In [584]:
corpus = Corpus(tokenizer, batch_size, seed_val)

********************************Data_precessing*********************************


convert_to_bert_train_dataset:   0%|                                                        | 0/103865 [00:00<?, ?it/s]

## dataset size is 115406
****************************Load OCNLI dataset ....*****************************


convert_to_bert_test_dataset: 100%|████████████████████████████████████████████| 25000/25000 [00:12<00:00, 2008.99it/s]


************************All Train and Test Data loaded !************************


In [586]:
train_loader, valid_loader, test_loader = corpus.get_loaders()

In [588]:
batch_size

4

In [587]:
for x in train_loader:
    print(x[:-1])
    print(x[-1])
    break

[tensor([[  101,  8356,  8160,  8162, 10658, 12404, 12570, 11173,   102,  8246,
          8160, 12873,  8162, 10658, 10541,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0, 

In [ ]:
x[:-1]

In [589]:
inputs = x[:-1]

In [593]:
inputs[0]

tensor([[  101,  8356,  8160,  8162, 10658, 12404, 12570, 11173,   102,  8246,
          8160, 12873,  8162, 10658, 10541,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [275]:
import random

In [282]:
a = np.random.rand(2, 3)

In [283]:
a

array([[0.66778585, 0.04161166, 0.86152516],
       [0.414541  , 0.90704763, 0.31789615]])

In [284]:
np.argmax(a, axis = 1)

array([2, 1], dtype=int64)

In [432]:
create_new_traindata(test_score, train_data, test_data)

In [595]:
class aa:
    
    def __init__(self):
        self.a = 1
        self.change()
    
    def change(self):
        self.b = 2 

In [596]:
b = aa()

In [598]:
b.b

2

In [4]:
import torch
import torch.nn as nn
x_input=torch.randn(3,3)#随机生成输入 
print('x_input:\n',x_input) 
y_target=torch.tensor([1,2,0])#设置输出具体值 print('y_target\n',y_target)

#计算输入softmax，此时可以看到每一行加到一起结果都是1
softmax_func=nn.Softmax(dim=1)
soft_output=softmax_func(x_input)
print('soft_output:\n',soft_output)

#在softmax的基础上取log
log_output=torch.log(soft_output)
print('log_output:\n',log_output)

#对比softmax与log的结合与nn.LogSoftmaxloss(负对数似然损失)的输出结果，发现两者是一致的。
logsoftmax_func=nn.LogSoftmax(dim=1)
logsoftmax_output=logsoftmax_func(x_input)
print('logsoftmax_output:\n',logsoftmax_output)

#pytorch中关于NLLLoss的默认参数配置为：reducetion=True、size_average=True
nllloss_func=nn.NLLLoss()
nlloss_output=nllloss_func(logsoftmax_output,y_target)
print('nlloss_output:\n',nlloss_output)

#直接使用pytorch中的loss_func=nn.CrossEntropyLoss()看与经过NLLLoss的计算是不是一样
crossentropyloss=nn.CrossEntropyLoss(weight = torch.tensor([0.4, 0.6, 1.0]))
crossentropyloss_output=crossentropyloss(x_input,y_target)
print('crossentropyloss_output:\n',crossentropyloss_output)

x_input:
 tensor([[-0.6854,  0.6218,  0.7670],
        [-0.8204, -0.2649, -1.2816],
        [ 0.3827,  0.5260, -0.1332]])
soft_output:
 tensor([[0.1115, 0.4121, 0.4764],
        [0.2964, 0.5166, 0.1869],
        [0.3635, 0.4195, 0.2170]])
log_output:
 tensor([[-2.1937, -0.8866, -0.7414],
        [-1.2159, -0.6604, -1.6771],
        [-1.0120, -0.8687, -1.5278]])
logsoftmax_output:
 tensor([[-2.1937, -0.8866, -0.7414],
        [-1.2159, -0.6604, -1.6771],
        [-1.0120, -0.8687, -1.5278]])
nlloss_output:
 tensor(1.1919)
crossentropyloss_output:
 tensor(1.3069)


In [9]:
print(f"{'classification_report':*^80}")

*****************************classification_report******************************


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
docs = [
    "it is a good day, I like to stay here",
    "I am happy to be here",
    "I am bob",
    "it is sunny today",
    "I have a party today",
    "it is a dog and that is a cat",
    "there are dog and cat on the tree",
    "I study hard this morning",
    "today is a good day",
    "tomorrow will be a good day",
    "I like coffee, I like book and I like apple",
    "I do not like it",
    "I am kitty, I like bob",
    "I do not care who like bob, but I like kitty",
    "It is coffee time, bring your cup",
]

In [12]:
vectorizer = TfidfVectorizer()

In [14]:
tf_idf = vectorizer.fit_transform(docs)

In [15]:
tf_idf

<15x44 sparse matrix of type '<class 'numpy.float64'>'
	with 77 stored elements in Compressed Sparse Row format>

In [16]:
print("idf: ", [(n, idf) for idf, n in zip(vectorizer.idf_, vectorizer.get_feature_names())])

idf:  [('am', 2.386294361119891), ('and', 2.386294361119891), ('apple', 3.0794415416798357), ('are', 3.0794415416798357), ('be', 2.6739764335716716), ('bob', 2.386294361119891), ('book', 3.0794415416798357), ('bring', 3.0794415416798357), ('but', 3.0794415416798357), ('care', 3.0794415416798357), ('cat', 2.6739764335716716), ('coffee', 2.6739764335716716), ('cup', 3.0794415416798357), ('day', 2.386294361119891), ('do', 2.6739764335716716), ('dog', 2.6739764335716716), ('good', 2.386294361119891), ('happy', 3.0794415416798357), ('hard', 3.0794415416798357), ('have', 3.0794415416798357), ('here', 2.6739764335716716), ('is', 1.9808292530117262), ('it', 1.9808292530117262), ('kitty', 2.6739764335716716), ('like', 1.9808292530117262), ('morning', 3.0794415416798357), ('not', 2.6739764335716716), ('on', 3.0794415416798357), ('party', 3.0794415416798357), ('stay', 3.0794415416798357), ('study', 3.0794415416798357), ('sunny', 3.0794415416798357), ('that', 3.0794415416798357), ('the', 3.0794415

In [17]:
len(vectorizer.idf_)

44

In [18]:
print("v2i: ", vectorizer.vocabulary_)

v2i:  {'it': 22, 'is': 21, 'good': 16, 'day': 13, 'like': 24, 'to': 37, 'stay': 29, 'here': 20, 'am': 0, 'happy': 17, 'be': 4, 'bob': 5, 'sunny': 31, 'today': 38, 'have': 19, 'party': 28, 'dog': 15, 'and': 1, 'that': 32, 'cat': 10, 'there': 34, 'are': 3, 'on': 27, 'the': 33, 'tree': 40, 'study': 30, 'hard': 18, 'this': 35, 'morning': 25, 'tomorrow': 39, 'will': 42, 'coffee': 11, 'book': 6, 'apple': 2, 'do': 14, 'not': 26, 'kitty': 23, 'care': 9, 'who': 41, 'but': 8, 'time': 36, 'bring': 7, 'your': 43, 'cup': 12}


In [27]:
q_words = ["I get a coffee cup", "I get a banana"]
for q in q_words:
    qtf_idf = vectorizer.transform([q])
    res = cosine_similarity(tf_idf, qtf_idf)
    res = res.ravel().argsort()[-3:]
    print("\ntop 3 docs for '{}':\n{}".format(q, [docs[i] for i in res[::-1]]))


top 3 docs for 'I get a coffee cup':
['It is coffee time, bring your cup', 'I like coffee, I like book and I like apple', 'I do not care who like bob, but I like kitty']

top 3 docs for 'I get a banana':
['It is coffee time, bring your cup', 'I do not care who like bob, but I like kitty', 'I am kitty, I like bob']


In [26]:
res

array([[0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.21398863],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.56058105]])

In [21]:
res = res.ravel().argsort()[-3:]

In [22]:
res

array([13, 10, 14], dtype=int64)

In [29]:
tf_idf.

<15x44 sparse matrix of type '<class 'numpy.float64'>'
	with 77 stored elements in Compressed Sparse Row format>

In [32]:
import pandas as pd

In [34]:
train_data = pd.read_csv("./tianchi_datasets/track3_round1_train.tsv", sep="\t", header=None,
                                  quoting=3, encoding="utf-8", names=["sentence1", "sentence2", "labels"])

In [36]:
import numpy as np

In [38]:
np.unique(train_data["labels"], return_counts = True)

(array([0, 1], dtype=int64), array([63564, 36436], dtype=int64))

In [39]:
0.75/(0.85 + 0.75)

0.46875

In [40]:
1 - 0.46875

0.53125

In [95]:
1/0.75

1.3333333333333333

In [96]:
1/0.85

1.1764705882352942

In [100]:
1.33 / 1.18

1.1271186440677967

In [99]:
0.85/0.75

1.1333333333333333

In [ ]:
1: 0.75 0:0.85

In [42]:
vectorizer.idf_

array([2.38629436, 2.38629436, 3.07944154, 3.07944154, 2.67397643,
       2.38629436, 3.07944154, 3.07944154, 3.07944154, 3.07944154,
       2.67397643, 2.67397643, 3.07944154, 2.38629436, 2.67397643,
       2.67397643, 2.38629436, 3.07944154, 3.07944154, 3.07944154,
       2.67397643, 1.98082925, 1.98082925, 2.67397643, 1.98082925,
       3.07944154, 2.67397643, 3.07944154, 3.07944154, 3.07944154,
       3.07944154, 3.07944154, 3.07944154, 3.07944154, 3.07944154,
       3.07944154, 3.07944154, 2.67397643, 2.38629436, 3.07944154,
       3.07944154, 3.07944154, 3.07944154, 3.07944154])

In [85]:
import pandas as pd
import itertools
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

train_data = pd.read_csv("./tianchi_datasets/track3_round1_train.tsv", sep="\t", header=None,nrows = 100,
                                  quoting=3, encoding="utf-8", names=["sentence1", "sentence2", "labels"])
train_data["document"] = train_data["sentence1"].str.cat(train_data["sentence2"], sep=" ")
test_data = pd.read_csv("./tianchi_datasets/track3_round1_testA.tsv", sep="\t", header=None, nrows = 100,
                                 quoting=3, encoding="utf-8", names=["sentence1", "sentence2"])
test_data["document"] = test_data["sentence1"].str.cat(test_data["sentence2"], sep=" ")


docs = []
q_words = []
for word1, word2 in itertools.zip_longest(train_data["document"], test_data["document"]):
    docs.append(word1)
    if word2 is None:
        continue
    q_words.append(word2)

vectorizer = TfidfVectorizer()
tf_idf = vectorizer.fit_transform(docs)

tmp = []
for q in q_words:
    qtf_idf = vectorizer.transform([q])
    res = cosine_similarity(tf_idf, qtf_idf).ravel()
    score = max(res)
    idx = np.argmax(res)
    if score > 0.7:
        label = train_data["labels"][idx]
    else:
        label = 2
    tmp.append(label)
    
test_data["labels"] = tmp
test_data1 = test_data[test_data["labels"] < 2]
new_data = pd.merge(test_data1, train_data, how = "outer")
new_data.to_csv("./tianchi_datasets/track3_round1_newtrain1.tsv", sep="\t", header=None, index=None)

In [86]:
test_data["labels"] = tmp
test_data1 = test_data[test_data["labels"] < 2]

In [88]:
new_data = pd.merge(test_data1, train_data, how = "outer")

In [90]:
new_data.to_csv("./tianchi_datasets/track3_round1_newtrain1.tsv", sep="\t", header=None, index=None)

In [116]:
import json
import pandas as pd
import os

test_score = "tianchi_datasets/test_03_07.json"
train_data = "tianchi_datasets/track3_round1_train.tsv"
test_data = "tianchi_datasets/track3_round1_testA.tsv"
def create_new_traindata(test_score, train_data, test_data):
    tmp = []
    dir_path = os.getcwd()

    with open(os.path.join(dir_path, test_score), "r", encoding="utf-8") as f:
        for line in f:
            content = json.loads(line)
            score = content["label"]
            idx = content["id"]
            if score > 0.95:
                tmp.append([idx, 1])
            elif score < 0.05:
                tmp.append([idx, 0])
            else:
                tmp.append([idx, "unk"])

    with open(os.path.join(dir_path, test_data), "r", encoding="utf-8") as fr:
        with open("tianchi_datasets/track3_round1_testA_label1.tsv", "w", encoding="utf-8") as fw:
            data = fr.readlines()
            for i in range(len(data)):
                content = data[i].strip()
                label = tmp[i][1]
                idx = tmp[i][0]
                if label in [0, 1]:
                    lis = content + "\t" + str(label) + "\n"
                else:
                    lis = ""
                fw.write(lis)

    train = pd.read_csv(os.path.join(dir_path, train_data), sep="\t", header=None,
                             names=["sentence1", "sentence2", "labels"])
    test = pd.read_csv("./tianchi_datasets/track3_round1_testA_label1.tsv", sep="\t", header=None,
                            names=["sentence1", "sentence2", "labels"])

    new_data = pd.concat([train, test], axis=0)
    new_data.to_csv("./tianchi_datasets/track3_round1_newtrain1.tsv", sep="\t", header=None, index=None)


In [117]:
create_new_traindata(test_score, train_data, test_data)

In [111]:
import pandas as pd
import itertools
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from tqdm import tqdm


train_data = pd.read_csv("./tianchi_datasets/track3_round1_train.tsv", sep="\t", header=None,
                         quoting=3, encoding="utf-8", names=["sentence1", "sentence2", "labels"])
train_data["document"] = train_data["sentence1"].str.cat(train_data["sentence2"], sep=" ")
test_data = pd.read_csv("./tianchi_datasets/track3_round1_testA.tsv", sep="\t", header=None, 
                        quoting=3, encoding="utf-8", names=["sentence1", "sentence2"])
test_data["document"] = test_data["sentence1"].str.cat(test_data["sentence2"], sep=" ")

docs = []
q_words = []
for word1, word2 in itertools.zip_longest(train_data["document"], test_data["document"]):
    docs.append(word1)
    if word2 is None:
        continue
    q_words.append(word2)

vectorizer = TfidfVectorizer()
tf_idf = vectorizer.fit_transform(docs)

tmp = []
for q in tqdm(q_words):
    qtf_idf = vectorizer.transform([q])
    res = cosine_similarity(tf_idf, qtf_idf).ravel()
    score = max(res)
    idx = np.argmax(res)
    if score > 0.7:
        label = train_data["labels"][idx]
    else:
        label = 2
    tmp.append(label)

test_data["labels"] = tmp
test_data1 = test_data[test_data["labels"] < 2]
new_data = pd.concat([train_data, test_data1], axis = 0)
new_data = new_data[["sentence1", "sentence2", "labels"]]
new_data.to_csv("./tianchi_datasets/track3_round1_newtrain3.tsv", sep="\t", header=None, index=None)

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 604.37it/s]


In [113]:
new_data = new_data[["sentence1", "sentence2", "labels"]]

In [115]:
new_data.to_csv("./tianchi_datasets/track3_round1_newtrain3.tsv", sep="\t", header=None, index=None)

In [118]:
data1 = pd.read_csv("./tianchi_datasets/track3_round1_newtrain1.tsv", sep="\t", header=None,
                         quoting=3, encoding="utf-8", names=["sentence1", "sentence2", "labels"])

In [128]:
data2 = pd.read_csv("./tianchi_datasets/track3_round1_newtrain2.tsv", sep="\t", header=None,
                         quoting=3, encoding="utf-8", names=["sentence1", "sentence2", "labels"])

In [132]:
new_data = pd.merge(data1, data2, how = "outer")
new_data.to_csv("./tianchi_datasets/track3_round1_newtrain3.tsv", sep="\t", header=None, index=None)

In [138]:
new_data

,sentence1,sentence2,labels
0,1 2 3 4 5 6 7,8 9 10 4 11,0
1,12 13 14 15,12 15 11 16,0
2,17 18 12 19 20 21 22 23 24,12 23 25 6 26 27 19,1
3,28 29 30 31 11,32 33 34 30 31,1
4,29 35 36 29,29 37 36 29,1
...,...,...,...
121100,282 283 283 282 158 228 58 180,282 283 283 282 158 228 1645 56 58 180,0
121101,12 19 6788 23 24,12 19 307 3151 23,0
121102,72 29 146 146,176 29 227 134 146 146,1
121103,12 251 6277 6278 16,12 251 11350,1


In [130]:
data1

,sentence1,sentence2,labels
0,1 2 3 4 5 6 7,8 9 10 4 11,0
1,12 13 14 15,12 15 11 16,0
2,17 18 12 19 20 21 22 23 24,12 23 25 6 26 27 19,1
3,28 29 30 31 11,32 33 34 30 31,1
4,29 35 36 29,29 37 36 29,1
...,...,...,...
120894,4915 23 39 9 3544 19,158 478 426 19 1274 120 212 1378 157,0
120895,729 192 169 718 169 718 2061,20253 416 11,0
120896,29 81 140 72 12 5 1763 1764 12 459 16,1553 72 12 20811 6722 16,1
120897,740 2155 6667,20273 762 852 11,0


In [131]:
data2

,sentence1,sentence2,labels
0,1 2 3 4 5 6 7,8 9 10 4 11,0
1,12 13 14 15,12 15 11 16,0
2,17 18 12 19 20 21 22 23 24,12 23 25 6 26 27 19,1
3,28 29 30 31 11,32 33 34 30 31,1
4,29 35 36 29,29 37 36 29,1
...,...,...,...
100768,28 12 317 43 533,12 44 253 317 66,0
100769,2692 230 507 20516,12 168 230 507 16,0
100770,12 698 10 23 850 1575 315,12 698 850 1575 315,1
100771,522 2746 1277 1277 48 336,169 1192 522 6 1277 1277 48 336,1


In [139]:
pd.merge(data1, data2, how = "inner")

,sentence1,sentence2,labels
0,1 2 3 4 5 6 7,8 9 10 4 11,0
1,12 13 14 15,12 15 11 16,0
2,17 18 12 19 20 21 22 23 24,12 23 25 6 26 27 19,1
3,28 29 30 31 11,32 33 34 30 31,1
4,29 35 36 29,29 37 36 29,1
...,...,...,...
100562,28 12 317 43 533,12 44 253 317 66,0
100563,2692 230 507 20516,12 168 230 507 16,0
100564,12 698 10 23 850 1575 315,12 698 850 1575 315,1
100565,522 2746 1277 1277 48 336,169 1192 522 6 1277 1277 48 336,1


In [140]:
a = pd.merge(data1[["sentence1", "sentence2"]], data2[["sentence1", "sentence2"]], how = "inner")

,sentence1,sentence2
0,1 2 3 4 5 6 7,8 9 10 4 11
1,12 13 14 15,12 15 11 16
2,17 18 12 19 20 21 22 23 24,12 23 25 6 26 27 19
3,28 29 30 31 11,32 33 34 30 31
4,29 35 36 29,29 37 36 29
...,...,...
100667,28 12 317 43 533,12 44 253 317 66
100668,2692 230 507 20516,12 168 230 507 16
100669,12 698 10 23 850 1575 315,12 698 850 1575 315
100670,522 2746 1277 1277 48 336,169 1192 522 6 1277 1277 48 336


In [192]:
import TEXTCNN_Pytorch

In [194]:
reload(TEXTCNN_Pytorch)

ModuleNotFoundError: spec not found for the module 'TEXTCNN_Pytorch'

In [195]:
from TEXTCNN_Pytorch import TEXTCNN

In [144]:
from Data_generator_vocab import Corpus

In [146]:
batch_size = 4
seed_val = 2021

In [154]:
corpus = Corpus(batch_size, seed_val)

********************************Data_precessing*********************************
## dataset size is 121105
*******************************Load  dataset ....*******************************
## Load Datasets Consume 0:00:06 s ###
************************All Train and Test Data loaded !************************


In [153]:
from imp import reload
import Data_generator_vocab
reload(Data_generator_vocab)

<module 'Data_generator_vocab' from 'E:\\lv_python\\NLP\\阿里天池小布助手\\Data_generator_vocab.py'>

In [156]:
for x in 

{0: '[PAD]',
 1: '[CLS]',
 2: '[UNK]',
 3: '[SEP]',
 4: '12',
 5: '29',
 6: '19',
 7: '23',
 8: '16',
 9: '11',
 10: '10',
 11: '9',
 12: '32',
 13: '39',
 14: '161',
 15: '126',
 16: '48',
 17: '243',
 18: '43',
 19: '106',
 20: '5',
 21: '6',
 22: '72',
 23: '66',
 24: '8',
 25: '13',
 26: '283',
 27: '282',
 28: '459',
 29: '28',
 30: '370',
 31: '317',
 32: '62',
 33: '76',
 34: '247',
 35: '140',
 36: '276',
 37: '50',
 38: '46',
 39: '169',
 40: '80',
 41: '300',
 42: '47',
 43: '202',
 44: '59',
 45: '217',
 46: '431',
 47: '45',
 48: '127',
 49: '134',
 50: '426',
 51: '536',
 52: '67',
 53: '447',
 54: '230',
 55: '24',
 56: '133',
 57: '522',
 58: '52',
 59: '533',
 60: '432',
 61: '227',
 62: '251',
 63: '226',
 64: '253',
 65: '453',
 66: '168',
 67: '358',
 68: '629',
 69: '25',
 70: '272',
 71: '176',
 72: '267',
 73: '102',
 74: '55',
 75: '462',
 76: '153',
 77: '79',
 78: '421',
 79: '173',
 80: '130',
 81: '334',
 82: '347',
 83: '101',
 84: '524',
 85: '525',
 86: '3

In [152]:
len(corpus.v2i)

20604

In [148]:
corpus.vocab

['[PAD]',
 '[CLS]',
 '[UNK]',
 '[SEP]',
 '12',
 '29',
 '19',
 '23',
 '16',
 '11',
 '10',
 '9',
 '32',
 '39',
 '161',
 '126',
 '48',
 '243',
 '43',
 '106',
 '5',
 '6',
 '72',
 '66',
 '8',
 '13',
 '283',
 '282',
 '459',
 '28',
 '370',
 '317',
 '62',
 '76',
 '247',
 '140',
 '276',
 '50',
 '46',
 '169',
 '80',
 '300',
 '47',
 '202',
 '59',
 '217',
 '431',
 '45',
 '127',
 '134',
 '426',
 '536',
 '67',
 '447',
 '230',
 '24',
 '133',
 '522',
 '52',
 '533',
 '432',
 '227',
 '251',
 '226',
 '253',
 '453',
 '168',
 '358',
 '629',
 '25',
 '272',
 '176',
 '267',
 '102',
 '55',
 '462',
 '153',
 '79',
 '421',
 '173',
 '130',
 '334',
 '347',
 '101',
 '524',
 '525',
 '335',
 '44',
 '360',
 '200',
 '592',
 '647',
 '163',
 '53',
 '485',
 '107',
 '254',
 '246',
 '128',
 '433',
 '574',
 '304',
 '142',
 '478',
 '344',
 '146',
 '70',
 '518',
 '275',
 '58',
 '180',
 '725',
 '229',
 '177',
 '122',
 '451',
 '248',
 '312',
 '263',
 '882',
 '56',
 '232',
 '212',
 '114',
 '444',
 '68',
 '470',
 '1006',
 '481',
 '

In [158]:
class_num = 2
maxlen = corpus.max_len
max_features = len(corpus.vocab)
embedding_dims = 300
epochs = 5
dropout_rate = 0.2
feature_size = 128
kernel_sizes = [2, 3, 4, 5] 

In [159]:
model = TEXTCNN(maxlen = maxlen,
                max_features = max_features,
                embedding_dims = embedding_dims,
                class_num = class_num,
                kernel_sizes = kernel_sizes,
                dropout_rate = dropout_rate,
                feature_size = feature_size)

In [160]:
train_loader, valid_loader, test_loader = corpus.get_loaders()

In [162]:
for x in train_loader:
    inputs = x[:-1]
    y = x[-1]
    break

In [183]:
model(*inputs)

tensor([[-0.0427,  1.7961],
        [-0.2734,  0.8580],
        [-0.1631,  0.8901],
        [-0.7502,  0.1731]], grad_fn=<AddmmBackward>)

In [188]:
import torch

In [189]:
torch.cuda.device_count()

1

In [2]:
import pandas as pd

In [3]:
import tensorflow as tf
train_data = pd.read_csv("../tianchi_datasets/track3_round1_newtrain3.tsv", sep="\t", header=None,
                                 names=["sentence1", "sentence2", "labels"])
train_data["document"] = train_data["sentence1"].str.cat(train_data["sentence2"], sep = " ")
test_data = pd.read_csv("../tianchi_datasets/track3_round1_testA.tsv", sep="\t", header=None,
                                names=["sentence1", "sentence2"])
test_data["document"] = test_data["sentence1"].str.cat(test_data["sentence2"], sep=" ")

FileNotFoundError: [Errno 2] No such file or directory: '../tianchi_datasets/track3_round1_newtrain3.tsv'

In [5]:
from Data_generator_vocab import Corpus
HYPERS = {
        "BATCH_SIZE":32 * 4,
        "LR": 5e-5,
        "EPOCHS": 7,
    }
seed_val = 2021
corpus = Corpus(HYPERS["BATCH_SIZE"], seed_val)
class_num = 2
maxlen = corpus.max_len
max_features = len(corpus.vocab)
embedding_dims = 400
dropout_rate = 0.2
feature_size = 1024
kernel_sizes = [2, 3, 4, 5]

********************************Data_precessing*********************************
## dataset size is 121105
*******************************Load  dataset ....*******************************
## Load Datasets Consume 0:00:07 s ###
************************All Train and Test Data loaded !************************


In [6]:
max_features

20604

In [15]:
from bert4keras.snippets import truncate_sequences

Using TensorFlow backend.


In [16]:
from bert4keras.tokenizers import load_vocab

In [23]:
import json

In [24]:
with open("./bert4keras/counts.json", "r") as fr:
    counts = json.load(fr)

In [18]:
min_count = 5
maxlen = 32
batch_size = 64

In [17]:
def load_data(filename):
    """加载数据
    单条格式：(文本1 ids, 文本2 ids, 标签id)
    """
    D = []
    with open(filename) as f:
        for l in f:
            l = l.strip().split('\t')
            if len(l) == 3:
                a, b, c = l[0], l[1], int(l[2])
            else:
                a, b, c = l[0], l[1], -5  # 未标注数据，标签为-5
            a = [int(i) for i in a.split(' ')]
            b = [int(i) for i in b.split(' ')]
            truncate_sequences(maxlen, -1, a, b)
            D.append((a, b, c))
    return D

In [21]:
data = load_data(
    './tianchi_datasets/track3_round1_newtrain3.tsv'
)
test_data = load_data("./tianchi_datasets/track3_round1_testA.tsv")

In [62]:
tokens = {}
for d in data + test_data:
    for i in d[0] + d[1]:
        tokens[i] = tokens.get(i, 0) + 1

tokens = {i: j for i, j in tokens.items() if j >= min_count}
tokens = sorted(tokens.items(), key=lambda s: -s[1])
#tokens = {
    #t[0]: i + 7
    #for i, t in enumerate(tokens)
#}

In [40]:
dict_path = "./pretrain_model/chinese_roberta_wwm_ext/vocab.txt"

In [41]:
with open("./bert4keras/counts.json", "r") as fr:
    counts = json.load(fr)
del counts['[CLS]']
del counts['[SEP]']
token_dict = load_vocab(dict_path)

In [43]:
freqs = [
    counts.get(i, 0) for i, j in sorted(token_dict.items(), key=lambda s: s[1])
]

In [46]:
import numpy as np

In [50]:
keep_tokens = list(np.argsort(freqs)[::-1])

In [53]:
len(tokens)

7512

In [56]:
tokens1 = {}
for d in data + test_data:
    for i in d[0] + d[1]:
        tokens1[i] = tokens1.get(i, 0) + 1

In [74]:
len(tokens)

7512

In [76]:
from bert4keras.models import build_transformer_model

In [77]:
?build_transformer_model

In [78]:
import json
import numpy as np
from sklearn.metrics import roc_auc_score
from bert4keras.backend import keras, K
from bert4keras.tokenizers import load_vocab
from bert4keras.models import build_transformer_model
from bert4keras.optimizers import Adam
from bert4keras.snippets import sequence_padding, DataGenerator
from bert4keras.snippets import truncate_sequences
from tqdm import tqdm

In [79]:
min_count = 5
maxlen = 32
batch_size = 64

In [80]:
def random_mask(text_ids):
    """随机mask
    """
    input_ids, output_ids = [], []
    rands = np.random.random(len(text_ids))
    for r, i in zip(rands, text_ids):
        if r < 0.15 * 0.8:
            input_ids.append(4)
            output_ids.append(i)
        elif r < 0.15 * 0.9:
            input_ids.append(i)
            output_ids.append(i)
        elif r < 0.15:
            input_ids.append(np.random.choice(len(tokens)) + 7)
            output_ids.append(i)
        else:
            input_ids.append(i)
            output_ids.append(0)
    return input_ids, output_ids


def sample_convert(text1, text2, label, random=False):
    """转换为MLM格式
    """
    text1_ids = [tokens.get(t, 1) for t in text1]
    text2_ids = [tokens.get(t, 1) for t in text2]
    if random:
        if np.random.random() < 0.5:
            text1_ids, text2_ids = text2_ids, text1_ids
        text1_ids, out1_ids = random_mask(text1_ids)
        text2_ids, out2_ids = random_mask(text2_ids)
    else:
        out1_ids = [0] * len(text1_ids)
        out2_ids = [0] * len(text2_ids)
    token_ids = [2] + text1_ids + [3] + text2_ids + [3]
    segment_ids = [0] * len(token_ids)
    output_ids = [label + 5] + out1_ids + [0] + out2_ids + [0]
    return token_ids, segment_ids, output_ids

In [81]:
class data_generator(DataGenerator):
    """数据生成器
    """
    def __iter__(self, random=False):
        batch_token_ids, batch_segment_ids, batch_output_ids = [], [], []
        for is_end, (text1, text2, label) in self.sample(random):
            token_ids, segment_ids, output_ids = sample_convert(
                text1, text2, label, random
            )
            batch_token_ids.append(token_ids)
            batch_segment_ids.append(segment_ids)
            batch_output_ids.append(output_ids)
            if len(batch_token_ids) == self.batch_size or is_end:
                batch_token_ids = sequence_padding(batch_token_ids)
                batch_segment_ids = sequence_padding(batch_segment_ids)
                batch_output_ids = sequence_padding(batch_output_ids)
                yield [batch_token_ids, batch_segment_ids], batch_output_ids
                batch_token_ids, batch_segment_ids, batch_output_ids = [], [], []

In [82]:
data_generator = data_generator(data, batch_size)

In [83]:
data_generator

In [86]:
import json
import numpy as np
from sklearn.metrics import roc_auc_score
from bert4keras.backend import keras, K
from bert4keras.tokenizers import load_vocab
from bert4keras.models import build_transformer_model
from bert4keras.optimizers import Adam
from bert4keras.snippets import sequence_padding, DataGenerator
from bert4keras.snippets import truncate_sequences
from tqdm import tqdm

min_count = 5
maxlen = 32
batch_size = 64

dict_path = "./pretrain_model/chinese_roberta_wwm_ext/vocab.txt"

def load_data(filename):
    """加载数据
    单条格式：(文本1 ids, 文本2 ids, 标签id)
    """
    D = []
    with open(filename) as f:
        for l in f:
            l = l.strip().split('\t')
            if len(l) == 3:
                a, b, c = l[0], l[1], int(l[2])
            else:
                a, b, c = l[0], l[1], -5  # 未标注数据，标签为-5
            a = [int(i) for i in a.split(' ')]
            b = [int(i) for i in b.split(' ')]
            truncate_sequences(maxlen, -1, a, b)
            D.append((a, b, c))
    return D
train_data = load_data(
    './tianchi_datasets/track3_round1_train.tsv'
)
test_data = load_data(
    './tianchi_datasets/track3_round1_testA.tsv'
)


def load_data(filename):
    """加载数据
    单条格式：(文本1 ids, 文本2 ids, 标签id)
    """
    D = []
    with open(filename) as f:
        for l in f:
            l = l.strip().split('\t')
            if len(l) == 3:
                a, b, c = l[0], l[1], int(l[2])
            else:
                a, b, c = l[0], l[1], -5  # 未标注数据，标签为-5
            a = [int(i) for i in a.split(' ')]
            b = [int(i) for i in b.split(' ')]
            truncate_sequences(maxlen, -1, a, b)
            D.append((a, b, c))
    return D

# 统计词频
tokens = {}
for d in data + test_data:
    for i in d[0] + d[1]:
        tokens[i] = tokens.get(i, 0) + 1

tokens = {i: j for i, j in tokens.items() if j >= min_count}
tokens = sorted(tokens.items(), key=lambda s: -s[1])

token_dict = load_vocab(dict_path)
counts = json.load(open('./bert4keras/counts.json'))
del counts['[CLS]']
del counts['[SEP]']
keep_tokens = {j:counts.get(i, 0) for i, j in sorted(token_dict.items(), key = lambda s:s[1])}
keep_tokens = sorted(keep_tokens.items(), key = lambda s:-s[1])
keep_tokens1 = keep_tokens[:len(tokens)]
v2i = {tokens[i][0]:keep_tokens1[i][0] for i in range(len(tokens))}


"""
with open("v2i.json", "w") as fw:
    json.dump(v2i, fw)

with open('test.json', 'r') as f:
    data = json.load(f)
"""

'\nwith open("v2i.json", "w") as fw:\n    json.dump(v2i, fw)\n\nwith open(\'test.json\', \'r\') as f:\n    data = json.load(f)\n'

In [88]:
len(tokens)

7512

In [94]:
v2i['[UNK]'] = 100
v2i['[CLS]'] = 101
v2i['[SEP]'] = 102

In [96]:
len(v2i)

7515

In [93]:
for i,j in v2i.items():
    if j in [100, 101, 102, 103]:
        print(i)

In [97]:
with open("v2i.json", "w") as fw:
    json.dump(v2i, fw)

In [99]:
with open('v2i.json', 'r') as f:
    data = json.load(f)

In [101]:
len(data)

7515

In [108]:
# %load finetune_test.py
#!/usr/bin/env python
"""
FileName: finetune.py
Description:
Author: Stark Lv
Date: 2021/2/27 6:13 PM
Version: 0.1
"""

from transformers import get_linear_schedule_with_warmup
from models import MODELS
import torch
import logging
from torch import nn
#from Data_generator import Corpus
from Data_generator_vocab import Corpus
#from Data_generator_tokenizer import Corpus
import time
import datetime
from tqdm import tqdm
import numpy as np
import torch.nn.functional as F
import os
import json
import pandas as pd
from sklearn.metrics import roc_auc_score


def format_time(elapsed):
    elapsed_rounded = int(round(elapsed))
    return str(datetime.timedelta(seconds = elapsed_rounded))

class fintune():

    def __init__(self, model_name = "BertBaseLinear", retrain_model_path = None):

        #retrain_model_path,是否重新开始训练,若从已训练好的模型继续训练,则输入模型,否则为None
        self.retrain_model_path = retrain_model_path
        self.model_name = model_name
        self.set_device() #创建了self.device, self.n_gpu
        self.set_random_seed() #设置随机数种子,保证所有结果可以复现


        #加载model
        self.model, self.tokenizer = self.load_model_tokenizer(model_name)

        # move model to GPU
        if self.n_gpu > 1:
            device_ids = [0, 1, 2, 3]
            self.model =torch.nn.DataParallel(self.model, device_ids= device_ids)
        self.model = self.model.to(self.device)

        #loss function & optimizer
        # 带权重的损失函数, 若positive_label为1的f1_score为 0.75，positive_label为0的f1_score为0.85
        #self.loss_func = nn.CrossEntropyLoss(weight= torch.tensor([1.1765, 1.3333])).to(self.device)
        self.loss_func = nn.CrossEntropyLoss(weight=torch.tensor([0.5, 0.5])).to(self.device)
        self.optimizer = torch.optim.Adam(
                self.model.parameters(),
                lr = HYPERS["LR"],
                eps = 1e-8)

        # 加载训练集、验证集、测试集
        print(f"{'加载训练集、验证集、测试集 Loading':*^80}")
        self.corpus = Corpus(HYPERS["BATCH_SIZE"], self.seed_val)
        self.train_loader, self.valid_loader, self.test_loader = self.corpus.get_loaders()



    def load_model_tokenizer(self, model_name):
        #model_path = os.path.join(os.getcwd(), MODELS[model_name]["path"])
        model_path = MODELS[model_name]["path"]
        assert model_name in MODELS

        if self.retrain_model_path is None:

            print(f"### 从 {model_path} 加载预训练模型, 重新训练 ###")
            model = MODELS[model_name]["class"](model_path)
        else:
            print(f"{'加载已预训练好的模型,继续训练':*^80}")
            model = torch.load("./fintuned_model/" + self.retrain_model_path)

        print(f"## Model {self.model_name} loaded. ##")

        tokenizer = MODELS[model_name]["tokenizer"].from_pretrained(model_path)

        ## __class__.__name__输出类的名字
        print(f"## Tokenizer {tokenizer.__class__.__name__} loaded. ##")

        return model, tokenizer

    def set_random_seed(self):
        # Set the seed value all over the place to make this reproducible.
        self.seed_val = 2021
        np.random.seed(self.seed_val)
        torch.manual_seed(self.seed_val)

    def set_device(self):
        ###设置日志时间的输出格式,说明在那段时间用过GPU
        LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"
        DATE_FORMAT = "%m/%d/%Y %H:%M:%S %p"
        logging.basicConfig(filename='NLP_GPU.log', level=logging.DEBUG, format=LOG_FORMAT, datefmt=DATE_FORMAT)

        # 设置运行环境GPU
        self.device = torch.device("cuda:0")
        self.n_gpu = torch.cuda.device_count()
        logging.info(msg = f"\n Using GPU, {self.n_gpu} device available")

    def run_epoch(self):

        #list to store a number of quantities such as
        #training and validation loss, validation accuracy, and timings.
        training_stats = []


        #Total number of training steps is [number of batches × number of epochs]
        total_steps = len(self.train_loader) * HYPERS["EPOCHS"]
        # create the learning rate scheduler
        self.scheduler = get_linear_schedule_with_warmup(
            self.optimizer, num_warmup_steps = 100, num_training_steps = total_steps
        )

        total_t0 = time.time()

        print(f"### Trainng {HYPERS['EPOCHS']} EPOCHS start ###")
        for epoch in tqdm(range(1, HYPERS["EPOCHS"] + 1), unit = "epoch",
                          desc = f"Training All {HYPERS['EPOCHS']} Epochs"):
            # Measure how long the per training epoch takes
            t0 = time.time()

            # Put the model into training model
            print(f"### EPOCH {epoch} train  start")
            self.model.train()

            epoch_train_loss = self.train(epoch)
            # measure avg batch loss in specific epoch
            avg_train_loss = epoch_train_loss / len(self.train_loader)
            # Measure how long this epoch took.
            training_time = format_time(time.time() - t0)
            print(f"## per batch train loss for epoch {epoch} is {avg_train_loss} ",
                  f"total train time : {training_time}")

            t0 = time.time()

            # eval mode
            # 在eval模式下不会应用DROPOUT和BATCHNORM
            print(f"### EPOCH {epoch} val start")
            self.model.eval()

            epoch_eval_loss, eval_auc = self.eval(metric=roc_auc_score)
            avg_eval_loss = epoch_eval_loss / len(self.valid_loader)
            validation_time = format_time(time.time() - t0)
            print(f"## per batch valid loss for epoch {epoch} is {avg_eval_loss} ",
                  f"total valid time : {validation_time}, Validation auc score {eval_auc}")

            # Record all statistics from this epoch.
            training_stats.append(
                {
                    'Epoch': epoch,
                    'Train Loss': epoch_train_loss,
                    'Avg Train Loss': avg_train_loss,
                    'Train Time': training_time,
                    'Valid Loss': epoch_eval_loss,
                    'Avg Valid Loss': avg_eval_loss,
                    'Valid Time': validation_time,
                    'Valid AUC': eval_auc,
                    'Test Results Dir': 'Model{}_BATCH{}_LR{}/'.format(self.model_name,
                     HYPERS['BATCH_SIZE'], HYPERS['LR'])
                }
            )

        test_t0 = time.time()
        self.test()
        print("Test Time Consume {}".format(format_time(time.time() - test_t0)))
        self.save_stats(training_stats)
        print(f"{'Training complete !':*^80}")
        print(f"### Total Process took {format_time(time.time() - total_t0)} ####")


    def train(self, epoch):
        total_train_loss = 0

        for batch in tqdm(self.train_loader, desc=f"Epoch {epoch}>>>>train", unit="batch"):
            # input_ids, attention_mask, token_type_id和Label混在一起
            inputs = [data.to(self.device) for data in batch[:-1]]
            labels = batch[-1].to(self.device)
            outputs = self.model(*inputs)


            batch_loss = self.loss_func(outputs, labels)
            total_train_loss += batch_loss.to("cpu").data.numpy()
            # clear any previously calculated gradients before performing a backward pass.
            self.optimizer.zero_grad()

            # perform a backward pass to calculate the gradients
            batch_loss.backward()

            # normalization of the gradients to 1.0 to avoid exploding gradients
            nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)

            # update parameters and take a step using the computed gradient
            self.optimizer.step()

            #update the learning rate 
            self.scheduler.step()

            #print(f"per batch loss {batch_loss.to('cpu').data.numpy()}")

        return total_train_loss

    def eval(self, metric):
        total_eval_loss = 0
        predicted_scores = np.array([])
        target_labels = np.array([])
        #predicted_labels = np.array([])

        for batch in self.valid_loader:
            inputs = [data.to(self.device) for data in batch[:-1]]
            labels = batch[-1].to(self.device)

            # Tell pytorch not to bother with constructing the compute graph during
            # the forward pass, since this is only needed for backprop (training).
            with torch.no_grad():
                outputs = self.model(*inputs)
                loss = self.loss_func(outputs, labels)
                total_eval_loss += loss.to("cpu").data.numpy()

                # move logits and labels to CPU,存储在CPU上面的变量不能和存储在GPU上面的变量进行运算
                logits = F.softmax(outputs.to("cpu"), dim = 1).data.numpy()
                label_ids = labels.to("cpu").data.numpy()
                y_pred = logits[:, 1]
                #y_pred1 = np.argmax(logits, axis = 1)
                predicted_scores = np.append(predicted_scores, y_pred)
                #predicted_labels = np.append(predicted_labels, y_pred1)
                target_labels = np.append(target_labels, label_ids)


        task_auc = metric(target_labels, predicted_scores)

        #print(f"{'classification_report':*^80}")
        #print(classification_report(target_labels, predicted_labels))
        return total_eval_loss , task_auc

    def test(self):

        prediction_scores = np.array([])
        print(f"{'Predict task':*^80}")
        for batch in tqdm(self.test_loader, desc = "Predict Scores Loading", unit = "batch"):
            inputs = [data.to(self.device) for data in batch]

            with torch.no_grad():
                outputs = self.model(*inputs)
                logits = F.softmax(outputs.to("cpu"), dim=1).data.numpy()
                y_pred = logits[:, 1]
                prediction_scores = np.append(prediction_scores, y_pred)

        self.save_predictions(prediction_scores)

    def save_predictions(self, predicts):
        file_path = "./finetuned_results/" + 'Model{}_BATCH{}_EPOCH{}_LR{}/'.format(self.model_name,
                  HYPERS['BATCH_SIZE'],HYPERS['EPOCHS'], HYPERS['LR'])
        if not os.path.exists(file_path):
            os.mkdir(file_path)

        #输出当前时间
        dt = str(datetime.datetime.now()).split(" ")[0]
        filename = "{}_{}_(对应id和score).json".format(self.model_name, dt)
        with open(file_path + filename, "w") as fw:
            for idx, predict in enumerate(predicts):
                predict = {"id": idx, "label": predict}
                json.dump(predict, fw)
                if idx == len(predicts) - 1:
                    break
                fw.write("\n")

        filename2 = "{}_{}_result.tsv".format(self.model_name, dt)
        with open(file_path + filename2, "w") as fw:
            for idx, predict in enumerate(predicts):
                fw.write(str(predict))
                if idx == len(predicts) - 1:
                    break
                fw.write("\n")

    def save_stats(self ,stats):
        df = pd.DataFrame(stats)
        filename = 'Stats_{}_BATCH{}_Epoch{}_LR{}.csv'.format(self.model_name, HYPERS['BATCH_SIZE'],
                            HYPERS['EPOCHS'], HYPERS['LR'])
        df.to_csv("./finetuned_results/" + filename, sep = ",", encoding="utf-8", index = False)

    def save_model(self):
        # 保存训练好的模型
        model_name = 'Model{}_BATCH{}_Epoch{}_LR{}_TIME{}.pkl'.format(self.model_name, HYPERS['BATCH_SIZE'],
        HYPERS['EPOCHS'], HYPERS['LR'],time.strftime("%Y_%m_%d_%H_%M",time.localtime()))
        torch.save(self.model, "./finetuned_model/" + model_name)
        print("Fintuned model saved")


if __name__ == "__main__":
    ## 使用编号为4,5的两张显卡
    #os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    #os.environ["CUDA_VISIBLE_DEVICES"] = "4, 5, 6, 7"
    # global hyper parameters
    HYPERS = {
        "BATCH_SIZE":4,
        "LR": 5e-5,
        "EPOCHS": 7,
    }
    app = fintune(model_name = "ChineseRobertaAttention", retrain_model_path = None)
    #app.run_epoch()
    #app.save_model()

### 从 ./pretrain_model/chinese_roberta_wwm_ext/ 加载预训练模型, 重新训练 ###
## Model ChineseRobertaAttention loaded. ##
## Tokenizer BertTokenizer loaded. ##
*****************************加载训练集、验证集、测试集 Loading******************************
********************************Data_precessing*********************************
## dataset size is 121105
*******************************Load  dataset ....*******************************
## Load Datasets Consume 0:00:06 s ###
************************All Train and Test Data loaded !************************


In [109]:
corpus = app.corpus

In [110]:
v2i = corpus.v2i

In [112]:
len(v2i)

7515

In [113]:
train_loader, valid_loader, test_loader = corpus.get_loaders()

In [114]:
for batch in train_loader:
    inputs = batch[:-1]
    y = batch[-1]
    break

In [ ]:
v2i['']

In [115]:
inputs

[tensor([[ 101, 6821, 1762,  782, 2157,  517,  100, 8969, 4638, 2496, 8024,  102,
          4638, 2496, 8024, 6121,  102,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0],
         [ 101, 8024, 1313, 4385, 1313,  102, 8024, 1313,  102,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,

In [1]:
import json
import numpy as np
from sklearn.metrics import roc_auc_score
from bert4keras.backend import keras, K
from bert4keras.tokenizers import load_vocab
from bert4keras.models import build_transformer_model
from bert4keras.optimizers import Adam
from bert4keras.snippets import sequence_padding, DataGenerator
from bert4keras.snippets import truncate_sequences
from tqdm import tqdm

min_count = 5
maxlen = 32
batch_size = 64

Using TensorFlow backend.


In [24]:
3 % 2

1

In [27]:
bin(11).replace("0b",'')

'1011'

In [29]:
import math
math.log(10**9, 2)

29.897352853986263

In [31]:
2 ** 29 > 10 ** 9

False

In [32]:
import json
import numpy as np
from sklearn.metrics import roc_auc_score
from bert4keras.backend import keras, K
from bert4keras.tokenizers import load_vocab
from bert4keras.models import build_transformer_model
from bert4keras.optimizers import Adam
from bert4keras.snippets import sequence_padding, DataGenerator
from bert4keras.snippets import truncate_sequences
from tqdm import tqdm

In [34]:
min_count = 5
maxlen = 32
batch_size = 4

In [35]:
def load_data(filename):
    """加载数据
    单条格式：(文本1 ids, 文本2 ids, 标签id)
    """
    D = []
    with open(filename) as f:
        for l in f:
            l = l.strip().split('\t')
            if len(l) == 3:
                a, b, c = l[0], l[1], int(l[2])
            else:
                a, b, c = l[0], l[1], -5  # 未标注数据，标签为-5
            a = [int(i) for i in a.split(' ')]
            b = [int(i) for i in b.split(' ')]
            truncate_sequences(maxlen, -1, a, b)
            D.append((a, b, c))
    return D

In [36]:
data = load_data(
    './tianchi_datasets/track3_round1_train.tsv'
)
train_data = [d for i, d in enumerate(data) if i % 10 != 0]
valid_data = [d for i, d in enumerate(data) if i % 10 == 0]
test_data = load_data(
    './tianchi_datasets/track3_round1_testA.tsv'
)

In [38]:
data[0]

([1, 2, 3, 4, 5, 6, 7], [8, 9, 10, 4, 11], 0)

In [39]:
class data_generator(DataGenerator):
    """数据生成器
    """
    def __iter__(self, random=False):
        batch_token_ids, batch_segment_ids, batch_output_ids = [], [], []
        for is_end, (text1, text2, label) in self.sample(random):
            token_ids, segment_ids, output_ids = sample_convert(
                text1, text2, label, random
            )
            batch_token_ids.append(token_ids)
            batch_segment_ids.append(segment_ids)
            batch_output_ids.append(output_ids)
            if len(batch_token_ids) == self.batch_size or is_end:
                batch_token_ids = sequence_padding(batch_token_ids)
                batch_segment_ids = sequence_padding(batch_segment_ids)
                batch_output_ids = sequence_padding(batch_output_ids)
                yield [batch_token_ids, batch_segment_ids], batch_output_ids
                batch_token_ids, batch_segment_ids, batch_output_ids = [], [], []

In [42]:
text_1, text2, label = data[0][0], data[0][1], data[0][2]

In [43]:
text_1

[1, 2, 3, 4, 5, 6, 7]

In [44]:
np.random.random(2)

array([0.80493773, 0.41796254])

In [45]:
input_ids , output_ids = [], [] 
rands = np.random.random(len(text_1))

In [55]:
for r, i in zip(rands, text_1):
    if r < 0.15 * 0.8:
        input_ids.append(4)
        output_ids.append(i)
    elif r < 0.15 * 0.9:
        input_ids.append(i)
        output_ids.append(i)
    elif r < 0.15:
        input_ids.append(np.random.choice(len(tokens)) + 7)
        output_ids.append(i)
    else:
        input_ids.append(i)
        output_ids.append(0)

In [56]:
input_ids

[1, 2, 3, 4, 4, 6, 7]

In [57]:
output_ids

[0, 0, 0, 0, 5, 0, 0]

In [1]:
import json
from bert4keras.tokenizers import load_vocab
from bert4keras.snippets import truncate_sequences

Using TensorFlow backend.


In [2]:
help(truncate_sequences)

Help on function truncate_sequences in module bert4keras.snippets:

truncate_sequences(maxlen, index, *sequences)
    截断总长度至不超过maxlen



In [3]:
min_count = 5
maxlen = 100

In [4]:
def load_data(filename):
    """加载数据
    单条格式：(文本1 ids, 文本2 ids, 标签id)
    """
    D = []
    with open(filename) as f:
        for l in f:
            l = l.strip().split('\t')
            if len(l) == 3:
                a, b, c = l[0], l[1], int(l[2])
            else:
                a, b, c = l[0], l[1], -5  # 未标注数据，标签为-5
            a = [int(i) for i in a.split(' ')]
            b = [int(i) for i in b.split(' ')]
            truncate_sequences(maxlen, -1, a, b)
            D.append((a, b, c))
    return D

In [5]:
train_data = load_data(
    './tianchi_datasets/track3_round1_train.tsv'
)
test_data = load_data(
    './tianchi_datasets/track3_round1_testA.tsv'
)
dict_path = "./pretrain_model/chinese_roberta_wwm_ext/vocab.txt"

In [8]:
tokens = {}
for d in train_data + test_data:
    for i in d[0] + d[1]:
        tokens[i] = tokens.get(i, 0) + 1

tokens = {i: j for i, j in tokens.items() if j >= min_count}
tokens = sorted(tokens.items(), key=lambda s: -s[1])

In [10]:
token_dict = load_vocab(dict_path)
with open('./counts.json', 'r') as f:
    counts = json.load(f)

In [12]:
del counts['[CLS]']
del counts['[SEP]']

In [26]:
keep_tokens = {i:counts.get(i, 0) for i, j in sorted(token_dict.items())}
keep_tokens1 = sorted(keep_tokens.items(), key = lambda s:-s[1])

In [30]:
keep_tokens1 = keep_tokens1[:len(tokens)]


In [35]:
v2i_chinese = {tokens[i][0]:keep_tokens1[i][0] for i in range(len(tokens))}

In [37]:
train_data[0]

([1, 2, 3, 4, 5, 6, 7], [8, 9, 10, 4, 11], 0)

In [40]:
len(v2i_chinese)

6925

In [54]:
import pandas as pd

In [55]:
train_data = pd.read_csv("./tianchi_datasets/track3_round1_newtrain3.tsv", sep="\t", header=None,
                                  quoting=3, encoding="utf-8", names=["sentence1", "sentence2", "labels"])

In [82]:
def convert_to_chinese(text_ids):
    s = "".join([v2i_chinese.get(int(num), '[PAD]') for num in text_ids.split(" ")])
    return s

In [89]:
train_data["sentence1_cn"] = train_data["sentence1"].apply(convert_to_chinese())

In [90]:
train_data

,sentence1,sentence2,labels,sentence1_cn
0,1 2 3 4 5 6 7,8 9 10 4 11,0,音术从政学国声
1,12 13 14 15,12 15 11 16,0,，以社毒
2,17 18 12 19 20 21 22 23 24,12 23 25 6 26 27 19,1,泽迎，。古固厂、多
3,28 29 30 31 11,32 33 34 30 31,1,用的测件是
4,29 35 36 29,29 37 36 29,1,的重领的
...,...,...,...,...
121100,282 283 283 282 158 228 58 180,282 283 283 282 158 228 1645 56 58 180,0,会个个会系数区建
121101,12 19 6788 23 24,12 19 307 3151 23,0,，。j、多
121102,72 29 146 146,176 29 227 134 146 146,1,：的性性
121103,12 251 6277 6278 16,12 251 11350,1,，经cleur一


In [61]:
token_dict['[PAD]']

0

In [63]:
[v2i_chinese[num] for num in [12, 13, 14, 15]]

array(['1 2 3 4 5 6 7', '12 13 14 15', '17 18 12 19 20 21 22 23 24', ...,
       '72 29 146 146', '12 251 6277 6278 16',
       '282 283 283 282 140 632 329 330 1105'], dtype=object)

In [87]:
[v2i_chinese[num] for num in [12, 13, 14, 15]]

['，', '以', '社', '毒']

In [100]:
with open("v2i.json", "w") as fw:
    json.dump(data1, fw)

In [98]:
data1["[PAD]"] = 0
data1["[MASK]"] = 103

In [99]:
len(data1)

7517

In [97]:
for i, j in data1.items():
    if j in [0, 100, 101, 102, 103]:
        print(i)

[UNK]
[CLS]
[SEP]


In [101]:
train_data = pd.read_csv("./tianchi_datasets/track3_round1_newtrain3.tsv", sep="\t", header=None,
                                 names=["sentence1", "sentence2", "labels"])
train_data["document"] = train_data["sentence1"].str.cat(train_data["sentence2"], sep = " [SEP] ")
train_data["document"] = train_data["document"].map(lambda x:[word for word in x.split(" ")])

In [102]:
train_data

,sentence1,sentence2,labels,document
0,1 2 3 4 5 6 7,8 9 10 4 11,0,"[1, 2, 3, 4, 5, 6, 7, [SEP], 8, 9, 10, 4, 11]"
1,12 13 14 15,12 15 11 16,0,"[12, 13, 14, 15, [SEP], 12, 15, 11, 16]"
2,17 18 12 19 20 21 22 23 24,12 23 25 6 26 27 19,1,"[17, 18, 12, 19, 20, 21, 22, 23, 24, [SEP], 12..."
3,28 29 30 31 11,32 33 34 30 31,1,"[28, 29, 30, 31, 11, [SEP], 32, 33, 34, 30, 31]"
4,29 35 36 29,29 37 36 29,1,"[29, 35, 36, 29, [SEP], 29, 37, 36, 29]"
...,...,...,...,...
121100,282 283 283 282 158 228 58 180,282 283 283 282 158 228 1645 56 58 180,0,"[282, 283, 283, 282, 158, 228, 58, 180, [SEP],..."
121101,12 19 6788 23 24,12 19 307 3151 23,0,"[12, 19, 6788, 23, 24, [SEP], 12, 19, 307, 315..."
121102,72 29 146 146,176 29 227 134 146 146,1,"[72, 29, 146, 146, [SEP], 176, 29, 227, 134, 1..."
121103,12 251 6277 6278 16,12 251 11350,1,"[12, 251, 6277, 6278, 16, [SEP], 12, 251, 11350]"


In [148]:
def _get_loss_mask(len_arange, seq, pad_id):
    rand_id = np.random.choice(len_arange, size=max(2, int(MASK_RATE * len(len_arange))), replace=False)
    loss_mask = np.full_like(seq, pad_id, dtype=np.bool)
    loss_mask[rand_id] = True
    return loss_mask[None, :], rand_id

In [149]:
def do_mask(seq, len_arange, pad_id, mask_id):
    loss_mask, rand_id = _get_loss_mask(len_arange, seq, pad_id)
    seq[rand_id] = mask_id
    return loss_mask

In [ ]:
def do_nothing(seq, len_arange, pad_id):
    loss_mask, _ = _get_loss_mask(len_arange, seq, pad_id)
    return loss_mask

In [ ]:
def do_replace(seq, len_arange, pad_id, word_ids):
    loss_mask, rand_id = _get_loss_mask(len_arange, seq, pad_id)
    seq[rand_id] = np.random.choice(word_ids, size=len(rand_id))
    return loss_mask

In [147]:
padded

array([ 101, 2099, 2990, 3309,  676, 2110, 1744, 5303,  102,  809,  782,
       3300,  676, 3221,  102,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0])

In [150]:
import numpy as np
input_ids = []
max_length = 93
for inputs in train_data["document"]:
    arange = np.arange(0, len(inputs) + 2)
    padded = np.full(max_length, fill_value=0, dtype=np.int32)
    padded[:(len(inputs) + 2)] = [v2i['[CLS]']] + [v2i.get(x, v2i['[UNK]']) for x in inputs] + [v2i['[SEP]']]
    p = np.random.random
    if p < 0.7:
        loss_mask = do_mask(padded, arange, v2i["[PAD]"], v2i["[MASK]"])
    
    break

In [151]:
padded

array([ 101, 2099, 2990, 3309,  676,  103, 1744, 5303,  102,  809,  782,
       3300,  676,  103,  102,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0])

In [123]:
v2i['[PAD]']

0

In [136]:
MASK_RATE = 0.15

In [137]:
seq = padded.copy()
len_arange = arange
pad_id = v2i["[PAD]"]
mask_id = v2i["[MASK]"]

In [140]:
do_mask(seq, len_arange, pad_id, mask_id)

array([[False, False, False, False,  True, False, False, False, False,
        False, False,  True, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False]])

In [146]:
int(15 * 0.15)

2

In [159]:
word_ids

array(['6140', '1260', '9865', ..., '1319', '4224', '8969'], dtype='<U6')